In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_log_error
from sklearn.preprocessing import LabelEncoder


In [2]:
df=pd.read_csv(r"/content/energy-pop-exposure-nuclear-plants-locations_plants.csv")

In [3]:
df.dropna(axis='columns', inplace=True)

In [4]:
df

,FID,Country,Plant,NumReactor,Latitude,Longitude,p90_1200,p00_1200,p10_1200,p90u_1200,...,p10r_600,p90_300,p00_300,p10_300,p90u_300,p00u_300,p10u_300,p90r_300,p00r_300,p10r_300
0,0,SWEDEN,AGESTA,1,59.206022,18.082872,187382000,188684000,188250000,121178000.0,...,8972550.0,5013240.0,5227700.0,5471110.0,3426880.0,3596030.0,3764920.0,1586350.0,1631670.0,1706190
1,1,SPAIN,ALMARAZ,2,39.808100,-5.696940,136675000,147718000,163429000,87539700.0,...,19453700.0,17756500.0,18187800.0,20185200.0,10986000.0,11415400.0,12689800.0,6770480.0,6772380.0,7495340
2,2,BRAZIL,ANGRA,3,-23.007857,-44.458098,99195200,113894000,127898000,67751500.0,...,18605600.0,39546400.0,44701700.0,50210600.0,32788500.0,37064600.0,41648300.0,6757940.0,7637110.0,8562300
3,3,UNITED STATES OF AMERICA,ARKANSAS ONE,2,35.310320,-93.231289,117830000,132729000,146482000,92609500.0,...,9498240.0,5603180.0,6226360.0,6866840.0,3779400.0,4198920.0,4633770.0,1823770.0,2027450.0,2233070
4,4,SPAIN,ASCO,2,41.200000,0.566670,271854000,287134000,308922000,190825000.0,...,17594700.0,14398500.0,15095600.0,16830700.0,11215400.0,11773600.0,13151300.0,3183180.0,3322050.0,3679470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,271,CHINA,YANGJIANG,3,21.708333,112.261111,557032000,629419000,679747000,250848000.0,...,86375000.0,68115100.0,87136800.0,93428600.0,41415700.0,53433600.0,57372000.0,26699400.0,33703200.0,36056500
272,272,UNITED STATES OF AMERICA,YANKEE,1,42.727839,-72.929108,123430000,131527000,145162000,103377000.0,...,9951640.0,35873400.0,37247000.0,41124300.0,32253500.0,33508100.0,37010000.0,3619890.0,3738910.0,4114260
273,273,"KOREA, REPUBLIC OF",YONGGWANG,6,35.411130,126.416190,649425000,710033000,750775000,404604000.0,...,35715500.0,42866100.0,46496100.0,48569700.0,36191700.0,39809100.0,41589500.0,6674350.0,6686990.0,6980200
274,274,UKRAINE,ZAPOROZHE,6,47.511809,34.585460,273432000,274834000,272561000,162035000.0,...,22931000.0,20624400.0,19361900.0,17990600.0,13838800.0,12978800.0,12067100.0,6785590.0,6383160.0,5923510


In [5]:
columns_to_drop = ['FID', 'Plant', 'NumReactor']
df.drop(columns=columns_to_drop, inplace=True)
df

,Country,Latitude,Longitude,p90_1200,p00_1200,p10_1200,p90u_1200,p00u_1200,p10u_1200,p90r_1200,...,p10r_600,p90_300,p00_300,p10_300,p90u_300,p00u_300,p10u_300,p90r_300,p00r_300,p10r_300
0,SWEDEN,59.206022,18.082872,187382000,188684000,188250000,121178000.0,122299000.0,122399000.0,66203700,...,8972550.0,5013240.0,5227700.0,5471110.0,3426880.0,3596030.0,3764920.0,1586350.0,1631670.0,1706190
1,SPAIN,39.808100,-5.696940,136675000,147718000,163429000,87539700.0,93943200.0,103704000.0,49135800,...,19453700.0,17756500.0,18187800.0,20185200.0,10986000.0,11415400.0,12689800.0,6770480.0,6772380.0,7495340
2,BRAZIL,-23.007857,-44.458098,99195200,113894000,127898000,67751500.0,77821400.0,87432200.0,31443800,...,18605600.0,39546400.0,44701700.0,50210600.0,32788500.0,37064600.0,41648300.0,6757940.0,7637110.0,8562300
3,UNITED STATES OF AMERICA,35.310320,-93.231289,117830000,132729000,146482000,92609500.0,104781000.0,115697000.0,25220200,...,9498240.0,5603180.0,6226360.0,6866840.0,3779400.0,4198920.0,4633770.0,1823770.0,2027450.0,2233070
4,SPAIN,41.200000,0.566670,271854000,287134000,308922000,190825000.0,200465000.0,215092000.0,81029500,...,17594700.0,14398500.0,15095600.0,16830700.0,11215400.0,11773600.0,13151300.0,3183180.0,3322050.0,3679470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,CHINA,21.708333,112.261111,557032000,629419000,679747000,250848000.0,287891000.0,309439000.0,306184000,...,86375000.0,68115100.0,87136800.0,93428600.0,41415700.0,53433600.0,57372000.0,26699400.0,33703200.0,36056500
272,UNITED STATES OF AMERICA,42.727839,-72.929108,123430000,131527000,145162000,103377000.0,110101000.0,121564000.0,20052600,...,9951640.0,35873400.0,37247000.0,41124300.0,32253500.0,33508100.0,37010000.0,3619890.0,3738910.0,4114260
273,"KOREA, REPUBLIC OF",35.411130,126.416190,649425000,710033000,750775000,404604000.0,440983000.0,464987000.0,244821000,...,35715500.0,42866100.0,46496100.0,48569700.0,36191700.0,39809100.0,41589500.0,6674350.0,6686990.0,6980200
274,UKRAINE,47.511809,34.585460,273432000,274834000,272561000,162035000.0,163493000.0,162347000.0,111397000,...,22931000.0,20624400.0,19361900.0,17990600.0,13838800.0,12978800.0,12067100.0,6785590.0,6383160.0,5923510


In [6]:

# Label Encoding
label_encoder = LabelEncoder()  # Create a LabelEncoder instance
categorical_cols = [ 'Country']  # Specify the categorical columns to be encoded

for col in categorical_cols:
    df[col + '_encoded'] = label_encoder.fit_transform(df[col])  # Apply label encoding to each column

print("Label Encoded Data:")
df

Label Encoded Data:


,Country,Latitude,Longitude,p90_1200,p00_1200,p10_1200,p90u_1200,p00u_1200,p10u_1200,p90r_1200,...,p90_300,p00_300,p10_300,p90u_300,p00u_300,p10u_300,p90r_300,p00r_300,p10r_300,Country_encoded
0,SWEDEN,59.206022,18.082872,187382000,188684000,188250000,121178000.0,122299000.0,122399000.0,66203700,...,5013240.0,5227700.0,5471110.0,3426880.0,3596030.0,3764920.0,1586350.0,1631670.0,1706190,28
1,SPAIN,39.808100,-5.696940,136675000,147718000,163429000,87539700.0,93943200.0,103704000.0,49135800,...,17756500.0,18187800.0,20185200.0,10986000.0,11415400.0,12689800.0,6770480.0,6772380.0,7495340,27
2,BRAZIL,-23.007857,-44.458098,99195200,113894000,127898000,67751500.0,77821400.0,87432200.0,31443800,...,39546400.0,44701700.0,50210600.0,32788500.0,37064600.0,41648300.0,6757940.0,7637110.0,8562300,3
3,UNITED STATES OF AMERICA,35.310320,-93.231289,117830000,132729000,146482000,92609500.0,104781000.0,115697000.0,25220200,...,5603180.0,6226360.0,6866840.0,3779400.0,4198920.0,4633770.0,1823770.0,2027450.0,2233070,33
4,SPAIN,41.200000,0.566670,271854000,287134000,308922000,190825000.0,200465000.0,215092000.0,81029500,...,14398500.0,15095600.0,16830700.0,11215400.0,11773600.0,13151300.0,3183180.0,3322050.0,3679470,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,CHINA,21.708333,112.261111,557032000,629419000,679747000,250848000.0,287891000.0,309439000.0,306184000,...,68115100.0,87136800.0,93428600.0,41415700.0,53433600.0,57372000.0,26699400.0,33703200.0,36056500,6
272,UNITED STATES OF AMERICA,42.727839,-72.929108,123430000,131527000,145162000,103377000.0,110101000.0,121564000.0,20052600,...,35873400.0,37247000.0,41124300.0,32253500.0,33508100.0,37010000.0,3619890.0,3738910.0,4114260,33
273,"KOREA, REPUBLIC OF",35.411130,126.416190,649425000,710033000,750775000,404604000.0,440983000.0,464987000.0,244821000,...,42866100.0,46496100.0,48569700.0,36191700.0,39809100.0,41589500.0,6674350.0,6686990.0,6980200,17
274,UKRAINE,47.511809,34.585460,273432000,274834000,272561000,162035000.0,163493000.0,162347000.0,111397000,...,20624400.0,19361900.0,17990600.0,13838800.0,12978800.0,12067100.0,6785590.0,6383160.0,5923510,31


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276 entries, 0 to 275
Data columns (total 58 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Country          276 non-null    object 
 1   Latitude         276 non-null    float64
 2   Longitude        276 non-null    float64
 3   p90_1200         276 non-null    int64  
 4   p00_1200         276 non-null    int64  
 5   p10_1200         276 non-null    int64  
 6   p90u_1200        276 non-null    float64
 7   p00u_1200        276 non-null    float64
 8   p10u_1200        276 non-null    float64
 9   p90r_1200        276 non-null    int64  
 10  p00r_1200        276 non-null    int64  
 11  p10r_1200        276 non-null    int64  
 12  p90_30           276 non-null    float64
 13  p00_30           276 non-null    float64
 14  p10_30           276 non-null    float64
 15  p90u_30          276 non-null    float64
 16  p00u_30          276 non-null    float64
 17  p10u_30         

In [8]:
X=df[["p90_1200","p00_1200","p10_1200"]]
X

,p90_1200,p00_1200,p10_1200
0,187382000,188684000,188250000
1,136675000,147718000,163429000
2,99195200,113894000,127898000
3,117830000,132729000,146482000
4,271854000,287134000,308922000
...,...,...,...
271,557032000,629419000,679747000
272,123430000,131527000,145162000
273,649425000,710033000,750775000
274,273432000,274834000,272561000


In [9]:
import numpy as np
import pandas as pd

def estimate_coef(x, y):
    # number of observations/points
    n = np.size(x)

    # mean of x and y vector
    m_x = np.mean(x)
    m_y = np.mean(y)

    # calculating cross-deviation and deviation about x
    SS_xy = np.sum(y*x) - n*m_y*m_x
    SS_xx = np.sum(x*x) - n*m_x*m_x

    # calculating regression coefficients
    b_1 = SS_xy / SS_xx
    b_0 = m_y - b_1*m_x

    return (b_0, b_1)

# Assuming you have a DataFrame named 'df' containing the columns p90_1200, p00_1200, and p10_1200

M = []
N = []

for i in range(0, 276):
    x = np.array([0.1, 0.15, 0.20])
    y = np.array([df.loc[i, 'p90_1200'], df.loc[i, 'p00_1200'], df.loc[i, 'p10_1200']])
    coef = estimate_coef(x, y)
        # Accessing the regression coefficients
    b_0, b_1 = coef
    print(f"Regression Coefficients for row {i}: b_0 = {b_0}, b_1 = {b_1}")
    M.append(b_0)
    N.append(b_1)

Regression Coefficients for row 0: b_0 = 186803333.33333334, b_1 = 8679999.999999968
Regression Coefficients for row 1: b_0 = 109143000.00000015, b_1 = 267539999.99999902
Regression Coefficients for row 2: b_0 = 70608200.00000016, b_1 = 287027999.9999989
Regression Coefficients for row 3: b_0 = 89369000.00000016, b_1 = 286519999.9999989
Regression Coefficients for row 4: b_0 = 233701333.33333352, b_1 = 370679999.9999986
Regression Coefficients for row 5: b_0 = 44300533.33333342, b_1 = 161239999.9999994
Regression Coefficients for row 6: b_0 = 354666833.33333343, b_1 = 226729999.99999917
Regression Coefficients for row 7: b_0 = 146591666.66666663, b_1 = -80039999.9999997
Regression Coefficients for row 8: b_0 = 334174000.00000006, b_1 = 126459999.99999954
Regression Coefficients for row 9: b_0 = 135709000.00000018, b_1 = 332979999.9999988
Regression Coefficients for row 10: b_0 = 285644666.66666675, b_1 = 79879999.9999997
Regression Coefficients for row 11: b_0 = 329247833.3333335, b_1 

In [10]:
p20_1200=[]
for j in range(0,276):
    y=M[j]+N[j]*0.25
    p20_1200.append(y)

In [11]:
p20_1200=np.fix(p20_1200)
p20_1200

array([1.88973333e+08, 1.76027999e+08, 1.42365199e+08, 1.60998999e+08,
       3.26371333e+08, 8.46105330e+07, 4.11349333e+08, 1.26581666e+08,
       3.65788999e+08, 2.18953999e+08, 3.05614666e+08, 3.96635333e+08,
       5.38374000e+07, 3.05137999e+08, 4.35906666e+08, 4.33829999e+08,
       1.98190333e+08, 2.76039000e+05, 3.76270333e+08, 1.54162666e+08,
       4.15294000e+08, 7.35279330e+07, 3.80831666e+08, 3.45151999e+08,
       2.03938999e+08, 3.11338999e+08, 3.96490999e+08, 2.06890333e+08,
       2.04790333e+08, 3.94534666e+08, 1.93838666e+08, 4.41982666e+08,
       1.55628866e+08, 1.76452666e+08, 1.81855666e+08, 2.04021333e+08,
       4.53056999e+08, 2.23660333e+08, 4.18998666e+08, 8.97854333e+08,
       2.94826333e+08, 6.20960333e+08, 2.43572666e+08, 7.68232333e+08,
       2.71011333e+08, 7.03092666e+08, 3.64281333e+08, 4.00614333e+08,
       3.87241999e+08, 2.01744999e+08, 2.32434666e+08, 4.70343660e+07,
       1.02384733e+08, 1.36467999e+08, 4.43616666e+08, 4.39727333e+08,
      

In [12]:
df['p20_1200LR'] = p20_1200.tolist()
df

,Country,Latitude,Longitude,p90_1200,p00_1200,p10_1200,p90u_1200,p00u_1200,p10u_1200,p90r_1200,...,p00_300,p10_300,p90u_300,p00u_300,p10u_300,p90r_300,p00r_300,p10r_300,Country_encoded,p20_1200LR
0,SWEDEN,59.206022,18.082872,187382000,188684000,188250000,121178000.0,122299000.0,122399000.0,66203700,...,5227700.0,5471110.0,3426880.0,3596030.0,3764920.0,1586350.0,1631670.0,1706190,28,188973333.0
1,SPAIN,39.808100,-5.696940,136675000,147718000,163429000,87539700.0,93943200.0,103704000.0,49135800,...,18187800.0,20185200.0,10986000.0,11415400.0,12689800.0,6770480.0,6772380.0,7495340,27,176027999.0
2,BRAZIL,-23.007857,-44.458098,99195200,113894000,127898000,67751500.0,77821400.0,87432200.0,31443800,...,44701700.0,50210600.0,32788500.0,37064600.0,41648300.0,6757940.0,7637110.0,8562300,3,142365199.0
3,UNITED STATES OF AMERICA,35.310320,-93.231289,117830000,132729000,146482000,92609500.0,104781000.0,115697000.0,25220200,...,6226360.0,6866840.0,3779400.0,4198920.0,4633770.0,1823770.0,2027450.0,2233070,33,160998999.0
4,SPAIN,41.200000,0.566670,271854000,287134000,308922000,190825000.0,200465000.0,215092000.0,81029500,...,15095600.0,16830700.0,11215400.0,11773600.0,13151300.0,3183180.0,3322050.0,3679470,27,326371333.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,CHINA,21.708333,112.261111,557032000,629419000,679747000,250848000.0,287891000.0,309439000.0,306184000,...,87136800.0,93428600.0,41415700.0,53433600.0,57372000.0,26699400.0,33703200.0,36056500,6,744780999.0
272,UNITED STATES OF AMERICA,42.727839,-72.929108,123430000,131527000,145162000,103377000.0,110101000.0,121564000.0,20052600,...,37247000.0,41124300.0,32253500.0,33508100.0,37010000.0,3619890.0,3738910.0,4114260,33,155104999.0
273,"KOREA, REPUBLIC OF",35.411130,126.416190,649425000,710033000,750775000,404604000.0,440983000.0,464987000.0,244821000,...,46496100.0,48569700.0,36191700.0,39809100.0,41589500.0,6674350.0,6686990.0,6980200,17,804760999.0
274,UKRAINE,47.511809,34.585460,273432000,274834000,272561000,162035000.0,163493000.0,162347000.0,111397000,...,19361900.0,17990600.0,13838800.0,12978800.0,12067100.0,6785590.0,6383160.0,5923510,31,272738000.0


In [13]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

# Assuming you have a DataFrame named 'df' containing the columns p90_1200, p00_1200, and p10_1200

A = []
B = []

for i in range(0, 276):
    x = np.array([[0.1], [0.15], [0.20]])
    y = np.array([df.loc[i, 'p90_1200'], df.loc[i, 'p00_1200'], df.loc[i, 'p10_1200']])

    # Create a linear regression model
    model = LinearRegression()
    model.fit(x, y)

    # Access the regression coefficients
    b_0 = model.intercept_
    b_1 = model.coef_[0]

    print(f"Regression Coefficients for row {i}: b_0 = {b_0}, b_1 = {b_1}")
    A.append(b_0)
    B.append(b_1)

Regression Coefficients for row 0: b_0 = 186803333.33333334, b_1 = 8680000.0
Regression Coefficients for row 1: b_0 = 109142999.99999999, b_1 = 267540000.00000012
Regression Coefficients for row 2: b_0 = 70608200.0, b_1 = 287028000.00000006
Regression Coefficients for row 3: b_0 = 89369000.0, b_1 = 286520000.00000006
Regression Coefficients for row 4: b_0 = 233701333.3333333, b_1 = 370680000.00000006
Regression Coefficients for row 5: b_0 = 44300533.33333332, b_1 = 161240000.00000006
Regression Coefficients for row 6: b_0 = 354666833.3333333, b_1 = 226730000.00000006
Regression Coefficients for row 7: b_0 = 146591666.66666666, b_1 = -80040000.00000001
Regression Coefficients for row 8: b_0 = 334174000.0, b_1 = 126460000.00000001
Regression Coefficients for row 9: b_0 = 135709000.0, b_1 = 332980000.0
Regression Coefficients for row 10: b_0 = 285644666.6666667, b_1 = 79880000.00000001
Regression Coefficients for row 11: b_0 = 329247833.3333333, b_1 = 269550000.00000006
Regression Coeffic

In [14]:
p20_1200LIB=[]
for j in range(0,276):
    y=A[j]+B[j]*0.25
    p20_1200LIB.append(y)
p20_1200LIB=np.fix(p20_1200LIB)
p20_1200LIB
#USING LIBRARY IS LIB
df['p20_1200LIB'] = p20_1200LIB.tolist()
df


,Country,Latitude,Longitude,p90_1200,p00_1200,p10_1200,p90u_1200,p00u_1200,p10u_1200,p90r_1200,...,p10_300,p90u_300,p00u_300,p10u_300,p90r_300,p00r_300,p10r_300,Country_encoded,p20_1200LR,p20_1200LIB
0,SWEDEN,59.206022,18.082872,187382000,188684000,188250000,121178000.0,122299000.0,122399000.0,66203700,...,5471110.0,3426880.0,3596030.0,3764920.0,1586350.0,1631670.0,1706190,28,188973333.0,188973333.0
1,SPAIN,39.808100,-5.696940,136675000,147718000,163429000,87539700.0,93943200.0,103704000.0,49135800,...,20185200.0,10986000.0,11415400.0,12689800.0,6770480.0,6772380.0,7495340,27,176027999.0,176028000.0
2,BRAZIL,-23.007857,-44.458098,99195200,113894000,127898000,67751500.0,77821400.0,87432200.0,31443800,...,50210600.0,32788500.0,37064600.0,41648300.0,6757940.0,7637110.0,8562300,3,142365199.0,142365200.0
3,UNITED STATES OF AMERICA,35.310320,-93.231289,117830000,132729000,146482000,92609500.0,104781000.0,115697000.0,25220200,...,6866840.0,3779400.0,4198920.0,4633770.0,1823770.0,2027450.0,2233070,33,160998999.0,160999000.0
4,SPAIN,41.200000,0.566670,271854000,287134000,308922000,190825000.0,200465000.0,215092000.0,81029500,...,16830700.0,11215400.0,11773600.0,13151300.0,3183180.0,3322050.0,3679470,27,326371333.0,326371333.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,CHINA,21.708333,112.261111,557032000,629419000,679747000,250848000.0,287891000.0,309439000.0,306184000,...,93428600.0,41415700.0,53433600.0,57372000.0,26699400.0,33703200.0,36056500,6,744780999.0,744781000.0
272,UNITED STATES OF AMERICA,42.727839,-72.929108,123430000,131527000,145162000,103377000.0,110101000.0,121564000.0,20052600,...,41124300.0,32253500.0,33508100.0,37010000.0,3619890.0,3738910.0,4114260,33,155104999.0,155105000.0
273,"KOREA, REPUBLIC OF",35.411130,126.416190,649425000,710033000,750775000,404604000.0,440983000.0,464987000.0,244821000,...,48569700.0,36191700.0,39809100.0,41589500.0,6674350.0,6686990.0,6980200,17,804760999.0,804761000.0
274,UKRAINE,47.511809,34.585460,273432000,274834000,272561000,162035000.0,163493000.0,162347000.0,111397000,...,17990600.0,13838800.0,12978800.0,12067100.0,6785590.0,6383160.0,5923510,31,272738000.0,272738000.0


In [15]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor

# Assuming you have a DataFrame named 'df' containing the columns p90_1200, p00_1200, and p10_1200

D = []

for i in range(0, 276):
    x = np.array([[0.1], [0.15], [0.20]])
    y = np.array([df.loc[i, 'p90_1200'], df.loc[i, 'p00_1200'], df.loc[i, 'p10_1200']])

    regressor = DecisionTreeRegressor(random_state=0)
    regressor.fit(x, y)
    y_pred = regressor.predict([[0.25]])
    D.append(y_pred)

In [16]:
D=np.fix(D)
df['P20_1200DT'] = D.tolist()
df

,Country,Latitude,Longitude,p90_1200,p00_1200,p10_1200,p90u_1200,p00u_1200,p10u_1200,p90r_1200,...,p90u_300,p00u_300,p10u_300,p90r_300,p00r_300,p10r_300,Country_encoded,p20_1200LR,p20_1200LIB,P20_1200DT
0,SWEDEN,59.206022,18.082872,187382000,188684000,188250000,121178000.0,122299000.0,122399000.0,66203700,...,3426880.0,3596030.0,3764920.0,1586350.0,1631670.0,1706190,28,188973333.0,188973333.0,[188250000.0]
1,SPAIN,39.808100,-5.696940,136675000,147718000,163429000,87539700.0,93943200.0,103704000.0,49135800,...,10986000.0,11415400.0,12689800.0,6770480.0,6772380.0,7495340,27,176027999.0,176028000.0,[163429000.0]
2,BRAZIL,-23.007857,-44.458098,99195200,113894000,127898000,67751500.0,77821400.0,87432200.0,31443800,...,32788500.0,37064600.0,41648300.0,6757940.0,7637110.0,8562300,3,142365199.0,142365200.0,[127898000.0]
3,UNITED STATES OF AMERICA,35.310320,-93.231289,117830000,132729000,146482000,92609500.0,104781000.0,115697000.0,25220200,...,3779400.0,4198920.0,4633770.0,1823770.0,2027450.0,2233070,33,160998999.0,160999000.0,[146482000.0]
4,SPAIN,41.200000,0.566670,271854000,287134000,308922000,190825000.0,200465000.0,215092000.0,81029500,...,11215400.0,11773600.0,13151300.0,3183180.0,3322050.0,3679470,27,326371333.0,326371333.0,[308922000.0]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,CHINA,21.708333,112.261111,557032000,629419000,679747000,250848000.0,287891000.0,309439000.0,306184000,...,41415700.0,53433600.0,57372000.0,26699400.0,33703200.0,36056500,6,744780999.0,744781000.0,[679747000.0]
272,UNITED STATES OF AMERICA,42.727839,-72.929108,123430000,131527000,145162000,103377000.0,110101000.0,121564000.0,20052600,...,32253500.0,33508100.0,37010000.0,3619890.0,3738910.0,4114260,33,155104999.0,155105000.0,[145162000.0]
273,"KOREA, REPUBLIC OF",35.411130,126.416190,649425000,710033000,750775000,404604000.0,440983000.0,464987000.0,244821000,...,36191700.0,39809100.0,41589500.0,6674350.0,6686990.0,6980200,17,804760999.0,804761000.0,[750775000.0]
274,UKRAINE,47.511809,34.585460,273432000,274834000,272561000,162035000.0,163493000.0,162347000.0,111397000,...,13838800.0,12978800.0,12067100.0,6785590.0,6383160.0,5923510,31,272738000.0,272738000.0,[272561000.0]


In [17]:
df.loc[:, 'P20_1200DT'] = D.tolist()


<ipython-input-17-df8a58fe72f3>:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, 'P20_1200DT'] = D.tolist()


In [18]:
df

,Country,Latitude,Longitude,p90_1200,p00_1200,p10_1200,p90u_1200,p00u_1200,p10u_1200,p90r_1200,...,p90u_300,p00u_300,p10u_300,p90r_300,p00r_300,p10r_300,Country_encoded,p20_1200LR,p20_1200LIB,P20_1200DT
0,SWEDEN,59.206022,18.082872,187382000,188684000,188250000,121178000.0,122299000.0,122399000.0,66203700,...,3426880.0,3596030.0,3764920.0,1586350.0,1631670.0,1706190,28,188973333.0,188973333.0,188250000.0
1,SPAIN,39.808100,-5.696940,136675000,147718000,163429000,87539700.0,93943200.0,103704000.0,49135800,...,10986000.0,11415400.0,12689800.0,6770480.0,6772380.0,7495340,27,176027999.0,176028000.0,163429000.0
2,BRAZIL,-23.007857,-44.458098,99195200,113894000,127898000,67751500.0,77821400.0,87432200.0,31443800,...,32788500.0,37064600.0,41648300.0,6757940.0,7637110.0,8562300,3,142365199.0,142365200.0,127898000.0
3,UNITED STATES OF AMERICA,35.310320,-93.231289,117830000,132729000,146482000,92609500.0,104781000.0,115697000.0,25220200,...,3779400.0,4198920.0,4633770.0,1823770.0,2027450.0,2233070,33,160998999.0,160999000.0,146482000.0
4,SPAIN,41.200000,0.566670,271854000,287134000,308922000,190825000.0,200465000.0,215092000.0,81029500,...,11215400.0,11773600.0,13151300.0,3183180.0,3322050.0,3679470,27,326371333.0,326371333.0,308922000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,CHINA,21.708333,112.261111,557032000,629419000,679747000,250848000.0,287891000.0,309439000.0,306184000,...,41415700.0,53433600.0,57372000.0,26699400.0,33703200.0,36056500,6,744780999.0,744781000.0,679747000.0
272,UNITED STATES OF AMERICA,42.727839,-72.929108,123430000,131527000,145162000,103377000.0,110101000.0,121564000.0,20052600,...,32253500.0,33508100.0,37010000.0,3619890.0,3738910.0,4114260,33,155104999.0,155105000.0,145162000.0
273,"KOREA, REPUBLIC OF",35.411130,126.416190,649425000,710033000,750775000,404604000.0,440983000.0,464987000.0,244821000,...,36191700.0,39809100.0,41589500.0,6674350.0,6686990.0,6980200,17,804760999.0,804761000.0,750775000.0
274,UKRAINE,47.511809,34.585460,273432000,274834000,272561000,162035000.0,163493000.0,162347000.0,111397000,...,13838800.0,12978800.0,12067100.0,6785590.0,6383160.0,5923510,31,272738000.0,272738000.0,272561000.0


In [19]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge

# Assuming you have a DataFrame named 'df' containing the columns p90_1200, p00_1200, and p10_1200

s = []

for i in range(0, 276):
    x = np.array([[0.1], [0.15], [0.20]])
    y = np.array([df.loc[i, 'p90_1200'], df.loc[i, 'p00_1200'], df.loc[i, 'p10_1200']])

    # Create a Ridge regression model
    regressor = Ridge(alpha=1.0)
    regressor.fit(x, y)

    x_test = np.array([[0.25]])
    y_pred = regressor.predict(x_test)
    s.append(y_pred)


In [20]:
df['P20_1200ridge'] = s
df

,Country,Latitude,Longitude,p90_1200,p00_1200,p10_1200,p90u_1200,p00u_1200,p10u_1200,p90r_1200,...,p00u_300,p10u_300,p90r_300,p00r_300,p10r_300,Country_encoded,p20_1200LR,p20_1200LIB,P20_1200DT,P20_1200ridge
0,SWEDEN,59.206022,18.082872,187382000,188684000,188250000,121178000.0,122299000.0,122399000.0,66203700,...,3596030.0,3764920.0,1586350.0,1631670.0,1706190,28,188973333.0,188973333.0,188250000.0,[188109651.74129355]
1,SPAIN,39.808100,-5.696940,136675000,147718000,163429000,87539700.0,93943200.0,103704000.0,49135800,...,11415400.0,12689800.0,6770480.0,6772380.0,7495340,27,176027999.0,176028000.0,163429000.0,[149407104.47761193]
2,BRAZIL,-23.007857,-44.458098,99195200,113894000,127898000,67751500.0,77821400.0,87432200.0,31443800,...,37064600.0,41648300.0,6757940.0,7637110.0,8562300,3,142365199.0,142365200.0,127898000.0,[113805200.0]
3,UNITED STATES OF AMERICA,35.310320,-93.231289,117830000,132729000,146482000,92609500.0,104781000.0,115697000.0,25220200,...,4198920.0,4633770.0,1823770.0,2027450.0,2233070,33,160998999.0,160999000.0,146482000.0,[132489547.26368159]
4,SPAIN,41.200000,0.566670,271854000,287134000,308922000,190825000.0,200465000.0,215092000.0,81029500,...,11773600.0,13151300.0,3183180.0,3322050.0,3679470,27,326371333.0,326371333.0,308922000.0,[289487751.2437811]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,CHINA,21.708333,112.261111,557032000,629419000,679747000,250848000.0,287891000.0,309439000.0,306184000,...,53433600.0,57372000.0,26699400.0,33703200.0,36056500,6,744780999.0,744781000.0,679747000.0,[622676522.3880597]
272,UNITED STATES OF AMERICA,42.727839,-72.929108,123430000,131527000,145162000,103377000.0,110101000.0,121564000.0,20052600,...,33508100.0,37010000.0,3619890.0,3738910.0,4114260,33,155104999.0,155105000.0,145162000.0,[133481119.40298507]
273,"KOREA, REPUBLIC OF",35.411130,126.416190,649425000,710033000,750775000,404604000.0,440983000.0,464987000.0,244821000,...,39809100.0,41589500.0,6674350.0,6686990.0,6980200,17,804760999.0,804761000.0,750775000.0,[703915228.8557214]
274,UKRAINE,47.511809,34.585460,273432000,274834000,272561000,162035000.0,163493000.0,162347000.0,111397000,...,12978800.0,12067100.0,6785590.0,6383160.0,5923510,31,272738000.0,272738000.0,272561000.0,[273604666.6666667]


In [21]:
df.loc[:, 'P20_1200ridge'] = s


<ipython-input-21-ae2148f48e4c>:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, 'P20_1200ridge'] = s


In [22]:
df

,Country,Latitude,Longitude,p90_1200,p00_1200,p10_1200,p90u_1200,p00u_1200,p10u_1200,p90r_1200,...,p00u_300,p10u_300,p90r_300,p00r_300,p10r_300,Country_encoded,p20_1200LR,p20_1200LIB,P20_1200DT,P20_1200ridge
0,SWEDEN,59.206022,18.082872,187382000,188684000,188250000,121178000.0,122299000.0,122399000.0,66203700,...,3596030.0,3764920.0,1586350.0,1631670.0,1706190,28,188973333.0,188973333.0,188250000.0,1.881097e+08
1,SPAIN,39.808100,-5.696940,136675000,147718000,163429000,87539700.0,93943200.0,103704000.0,49135800,...,11415400.0,12689800.0,6770480.0,6772380.0,7495340,27,176027999.0,176028000.0,163429000.0,1.494071e+08
2,BRAZIL,-23.007857,-44.458098,99195200,113894000,127898000,67751500.0,77821400.0,87432200.0,31443800,...,37064600.0,41648300.0,6757940.0,7637110.0,8562300,3,142365199.0,142365200.0,127898000.0,1.138052e+08
3,UNITED STATES OF AMERICA,35.310320,-93.231289,117830000,132729000,146482000,92609500.0,104781000.0,115697000.0,25220200,...,4198920.0,4633770.0,1823770.0,2027450.0,2233070,33,160998999.0,160999000.0,146482000.0,1.324895e+08
4,SPAIN,41.200000,0.566670,271854000,287134000,308922000,190825000.0,200465000.0,215092000.0,81029500,...,11773600.0,13151300.0,3183180.0,3322050.0,3679470,27,326371333.0,326371333.0,308922000.0,2.894878e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,CHINA,21.708333,112.261111,557032000,629419000,679747000,250848000.0,287891000.0,309439000.0,306184000,...,53433600.0,57372000.0,26699400.0,33703200.0,36056500,6,744780999.0,744781000.0,679747000.0,6.226765e+08
272,UNITED STATES OF AMERICA,42.727839,-72.929108,123430000,131527000,145162000,103377000.0,110101000.0,121564000.0,20052600,...,33508100.0,37010000.0,3619890.0,3738910.0,4114260,33,155104999.0,155105000.0,145162000.0,1.334811e+08
273,"KOREA, REPUBLIC OF",35.411130,126.416190,649425000,710033000,750775000,404604000.0,440983000.0,464987000.0,244821000,...,39809100.0,41589500.0,6674350.0,6686990.0,6980200,17,804760999.0,804761000.0,750775000.0,7.039152e+08
274,UKRAINE,47.511809,34.585460,273432000,274834000,272561000,162035000.0,163493000.0,162347000.0,111397000,...,12978800.0,12067100.0,6785590.0,6383160.0,5923510,31,272738000.0,272738000.0,272561000.0,2.736047e+08


In [23]:
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor



# Create a new column 'Predicted_p20_1200' and set it to 0
df['Predicted_p20_1200'] = 0

# Input features
X = df[['p90_1200', 'p00_1200', 'p10_1200']].values

# Target variable
y = df['p20_1200LR'].values

# Create a KNN regressor object
knn = KNeighborsRegressor(n_neighbors=5)

# Fit the model to the data
knn.fit(X, y)

# Make predictions for the input features
predictions = knn.predict(X)

# Assign the predicted values to the 'Predicted_p20_1200knn' column
df['Predicted_p20_1200knn'] = predictions


In [24]:
df

,Country,Latitude,Longitude,p90_1200,p00_1200,p10_1200,p90u_1200,p00u_1200,p10u_1200,p90r_1200,...,p90r_300,p00r_300,p10r_300,Country_encoded,p20_1200LR,p20_1200LIB,P20_1200DT,P20_1200ridge,Predicted_p20_1200,Predicted_p20_1200knn
0,SWEDEN,59.206022,18.082872,187382000,188684000,188250000,121178000.0,122299000.0,122399000.0,66203700,...,1586350.0,1631670.0,1706190,28,188973333.0,188973333.0,188250000.0,1.881097e+08,0,184029199.6
1,SPAIN,39.808100,-5.696940,136675000,147718000,163429000,87539700.0,93943200.0,103704000.0,49135800,...,6770480.0,6772380.0,7495340,27,176027999.0,176028000.0,163429000.0,1.494071e+08,0,176228332.6
2,BRAZIL,-23.007857,-44.458098,99195200,113894000,127898000,67751500.0,77821400.0,87432200.0,31443800,...,6757940.0,7637110.0,8562300,3,142365199.0,142365200.0,127898000.0,1.138052e+08,0,135754986.0
3,UNITED STATES OF AMERICA,35.310320,-93.231289,117830000,132729000,146482000,92609500.0,104781000.0,115697000.0,25220200,...,1823770.0,2027450.0,2233070,33,160998999.0,160999000.0,146482000.0,1.324895e+08,0,157149732.6
4,SPAIN,41.200000,0.566670,271854000,287134000,308922000,190825000.0,200465000.0,215092000.0,81029500,...,3183180.0,3322050.0,3679470,27,326371333.0,326371333.0,308922000.0,2.894878e+08,0,314587132.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,CHINA,21.708333,112.261111,557032000,629419000,679747000,250848000.0,287891000.0,309439000.0,306184000,...,26699400.0,33703200.0,36056500,6,744780999.0,744781000.0,679747000.0,6.226765e+08,0,730542466.0
272,UNITED STATES OF AMERICA,42.727839,-72.929108,123430000,131527000,145162000,103377000.0,110101000.0,121564000.0,20052600,...,3619890.0,3738910.0,4114260,33,155104999.0,155105000.0,145162000.0,1.334811e+08,0,154687199.4
273,"KOREA, REPUBLIC OF",35.411130,126.416190,649425000,710033000,750775000,404604000.0,440983000.0,464987000.0,244821000,...,6674350.0,6686990.0,6980200,17,804760999.0,804761000.0,750775000.0,7.039152e+08,0,821921466.0
274,UKRAINE,47.511809,34.585460,273432000,274834000,272561000,162035000.0,163493000.0,162347000.0,111397000,...,6785590.0,6383160.0,5923510,31,272738000.0,272738000.0,272561000.0,2.736047e+08,0,281498999.8


In [25]:
# Drop the 'knn2' and 'Predicted_p20_1200' columns
df = df.drop([ 'Predicted_p20_1200'], axis=1)
df

,Country,Latitude,Longitude,p90_1200,p00_1200,p10_1200,p90u_1200,p00u_1200,p10u_1200,p90r_1200,...,p10u_300,p90r_300,p00r_300,p10r_300,Country_encoded,p20_1200LR,p20_1200LIB,P20_1200DT,P20_1200ridge,Predicted_p20_1200knn
0,SWEDEN,59.206022,18.082872,187382000,188684000,188250000,121178000.0,122299000.0,122399000.0,66203700,...,3764920.0,1586350.0,1631670.0,1706190,28,188973333.0,188973333.0,188250000.0,1.881097e+08,184029199.6
1,SPAIN,39.808100,-5.696940,136675000,147718000,163429000,87539700.0,93943200.0,103704000.0,49135800,...,12689800.0,6770480.0,6772380.0,7495340,27,176027999.0,176028000.0,163429000.0,1.494071e+08,176228332.6
2,BRAZIL,-23.007857,-44.458098,99195200,113894000,127898000,67751500.0,77821400.0,87432200.0,31443800,...,41648300.0,6757940.0,7637110.0,8562300,3,142365199.0,142365200.0,127898000.0,1.138052e+08,135754986.0
3,UNITED STATES OF AMERICA,35.310320,-93.231289,117830000,132729000,146482000,92609500.0,104781000.0,115697000.0,25220200,...,4633770.0,1823770.0,2027450.0,2233070,33,160998999.0,160999000.0,146482000.0,1.324895e+08,157149732.6
4,SPAIN,41.200000,0.566670,271854000,287134000,308922000,190825000.0,200465000.0,215092000.0,81029500,...,13151300.0,3183180.0,3322050.0,3679470,27,326371333.0,326371333.0,308922000.0,2.894878e+08,314587132.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,CHINA,21.708333,112.261111,557032000,629419000,679747000,250848000.0,287891000.0,309439000.0,306184000,...,57372000.0,26699400.0,33703200.0,36056500,6,744780999.0,744781000.0,679747000.0,6.226765e+08,730542466.0
272,UNITED STATES OF AMERICA,42.727839,-72.929108,123430000,131527000,145162000,103377000.0,110101000.0,121564000.0,20052600,...,37010000.0,3619890.0,3738910.0,4114260,33,155104999.0,155105000.0,145162000.0,1.334811e+08,154687199.4
273,"KOREA, REPUBLIC OF",35.411130,126.416190,649425000,710033000,750775000,404604000.0,440983000.0,464987000.0,244821000,...,41589500.0,6674350.0,6686990.0,6980200,17,804760999.0,804761000.0,750775000.0,7.039152e+08,821921466.0
274,UKRAINE,47.511809,34.585460,273432000,274834000,272561000,162035000.0,163493000.0,162347000.0,111397000,...,12067100.0,6785590.0,6383160.0,5923510,31,272738000.0,272738000.0,272561000.0,2.736047e+08,281498999.8


In [26]:
import pandas as pd
from sklearn.neural_network import MLPRegressor

# Input features
X = df[['p90_1200', 'p00_1200', 'p10_1200']].values

# Target variable
y = df['p20_1200LR'].values

# Create an ANN regressor object
regressor = MLPRegressor(hidden_layer_sizes=(100,), activation='relu', solver='adam', random_state=42)

# Fit the model to the data
regressor.fit(X, y)

# Make predictions for the input features
predictions = regressor.predict(X)

# Add the predictions as a new column in the DataFrame
df['ANN2'] = predictions


In [27]:
df

,Country,Latitude,Longitude,p90_1200,p00_1200,p10_1200,p90u_1200,p00u_1200,p10u_1200,p90r_1200,...,p90r_300,p00r_300,p10r_300,Country_encoded,p20_1200LR,p20_1200LIB,P20_1200DT,P20_1200ridge,Predicted_p20_1200knn,ANN2
0,SWEDEN,59.206022,18.082872,187382000,188684000,188250000,121178000.0,122299000.0,122399000.0,66203700,...,1586350.0,1631670.0,1706190,28,188973333.0,188973333.0,188250000.0,1.881097e+08,184029199.6,2.230734e+08
1,SPAIN,39.808100,-5.696940,136675000,147718000,163429000,87539700.0,93943200.0,103704000.0,49135800,...,6770480.0,6772380.0,7495340,27,176027999.0,176028000.0,163429000.0,1.494071e+08,176228332.6,1.765751e+08
2,BRAZIL,-23.007857,-44.458098,99195200,113894000,127898000,67751500.0,77821400.0,87432200.0,31443800,...,6757940.0,7637110.0,8562300,3,142365199.0,142365200.0,127898000.0,1.138052e+08,135754986.0,1.336105e+08
3,UNITED STATES OF AMERICA,35.310320,-93.231289,117830000,132729000,146482000,92609500.0,104781000.0,115697000.0,25220200,...,1823770.0,2027450.0,2233070,33,160998999.0,160999000.0,146482000.0,1.324895e+08,157149732.6,1.558810e+08
4,SPAIN,41.200000,0.566670,271854000,287134000,308922000,190825000.0,200465000.0,215092000.0,81029500,...,3183180.0,3322050.0,3679470,27,326371333.0,326371333.0,308922000.0,2.894878e+08,314587132.6,3.427517e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,CHINA,21.708333,112.261111,557032000,629419000,679747000,250848000.0,287891000.0,309439000.0,306184000,...,26699400.0,33703200.0,36056500,6,744780999.0,744781000.0,679747000.0,6.226765e+08,730542466.0,7.319428e+08
272,UNITED STATES OF AMERICA,42.727839,-72.929108,123430000,131527000,145162000,103377000.0,110101000.0,121564000.0,20052600,...,3619890.0,3738910.0,4114260,33,155104999.0,155105000.0,145162000.0,1.334811e+08,154687199.4,1.579845e+08
273,"KOREA, REPUBLIC OF",35.411130,126.416190,649425000,710033000,750775000,404604000.0,440983000.0,464987000.0,244821000,...,6674350.0,6686990.0,6980200,17,804760999.0,804761000.0,750775000.0,7.039152e+08,821921466.0,8.299382e+08
274,UKRAINE,47.511809,34.585460,273432000,274834000,272561000,162035000.0,163493000.0,162347000.0,111397000,...,6785590.0,6383160.0,5923510,31,272738000.0,272738000.0,272561000.0,2.736047e+08,281498999.8,3.244027e+08


In [28]:
import pandas as pd
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import StandardScaler

# Input features
X = df[['p90_1200', 'p00_1200', 'p10_1200']].values
y = df['p20_1200LR'].values

# Perform feature scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Create an ElasticNet regressor object
regressor = ElasticNet(alpha=0.5, l1_ratio=0.5)

# Fit the model to the scaled data
regressor.fit(X_scaled, y)

# Make predictions for the scaled data
predictions = regressor.predict(X_scaled)

# Add the predictions as a new column in the DataFrame
df['en'] = predictions


In [29]:
df

,Country,Latitude,Longitude,p90_1200,p00_1200,p10_1200,p90u_1200,p00u_1200,p10u_1200,p90r_1200,...,p00r_300,p10r_300,Country_encoded,p20_1200LR,p20_1200LIB,P20_1200DT,P20_1200ridge,Predicted_p20_1200knn,ANN2,en
0,SWEDEN,59.206022,18.082872,187382000,188684000,188250000,121178000.0,122299000.0,122399000.0,66203700,...,1631670.0,1706190,28,188973333.0,188973333.0,188250000.0,1.881097e+08,184029199.6,2.230734e+08,2.132813e+08
1,SPAIN,39.808100,-5.696940,136675000,147718000,163429000,87539700.0,93943200.0,103704000.0,49135800,...,6772380.0,7495340,27,176027999.0,176028000.0,163429000.0,1.494071e+08,176228332.6,1.765751e+08,1.716552e+08
2,BRAZIL,-23.007857,-44.458098,99195200,113894000,127898000,67751500.0,77821400.0,87432200.0,31443800,...,7637110.0,8562300,3,142365199.0,142365200.0,127898000.0,1.138052e+08,135754986.0,1.336105e+08,1.327434e+08
3,UNITED STATES OF AMERICA,35.310320,-93.231289,117830000,132729000,146482000,92609500.0,104781000.0,115697000.0,25220200,...,2027450.0,2233070,33,160998999.0,160999000.0,146482000.0,1.324895e+08,157149732.6,1.558810e+08,1.532078e+08
4,SPAIN,41.200000,0.566670,271854000,287134000,308922000,190825000.0,200465000.0,215092000.0,81029500,...,3322050.0,3679470,27,326371333.0,326371333.0,308922000.0,2.894878e+08,314587132.6,3.427517e+08,3.253859e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,CHINA,21.708333,112.261111,557032000,629419000,679747000,250848000.0,287891000.0,309439000.0,306184000,...,33703200.0,36056500,6,744780999.0,744781000.0,679747000.0,6.226765e+08,730542466.0,7.319428e+08,6.930402e+08
272,UNITED STATES OF AMERICA,42.727839,-72.929108,123430000,131527000,145162000,103377000.0,110101000.0,121564000.0,20052600,...,3738910.0,4114260,33,155104999.0,155105000.0,145162000.0,1.334811e+08,154687199.4,1.579845e+08,1.540555e+08
273,"KOREA, REPUBLIC OF",35.411130,126.416190,649425000,710033000,750775000,404604000.0,440983000.0,464987000.0,244821000,...,6686990.0,6980200,17,804760999.0,804761000.0,750775000.0,7.039152e+08,821921466.0,8.299382e+08,7.813568e+08
274,UKRAINE,47.511809,34.585460,273432000,274834000,272561000,162035000.0,163493000.0,162347000.0,111397000,...,6383160.0,5923510,31,272738000.0,272738000.0,272561000.0,2.736047e+08,281498999.8,3.244027e+08,3.068750e+08


In [30]:
df = df.drop('Country', axis=1)
df

,Latitude,Longitude,p90_1200,p00_1200,p10_1200,p90u_1200,p00u_1200,p10u_1200,p90r_1200,p00r_1200,...,p00r_300,p10r_300,Country_encoded,p20_1200LR,p20_1200LIB,P20_1200DT,P20_1200ridge,Predicted_p20_1200knn,ANN2,en
0,59.206022,18.082872,187382000,188684000,188250000,121178000.0,122299000.0,122399000.0,66203700,66384500,...,1631670.0,1706190,28,188973333.0,188973333.0,188250000.0,1.881097e+08,184029199.6,2.230734e+08,2.132813e+08
1,39.808100,-5.696940,136675000,147718000,163429000,87539700.0,93943200.0,103704000.0,49135800,53774400,...,6772380.0,7495340,27,176027999.0,176028000.0,163429000.0,1.494071e+08,176228332.6,1.765751e+08,1.716552e+08
2,-23.007857,-44.458098,99195200,113894000,127898000,67751500.0,77821400.0,87432200.0,31443800,36072700,...,7637110.0,8562300,3,142365199.0,142365200.0,127898000.0,1.138052e+08,135754986.0,1.336105e+08,1.327434e+08
3,35.310320,-93.231289,117830000,132729000,146482000,92609500.0,104781000.0,115697000.0,25220200,27947300,...,2027450.0,2233070,33,160998999.0,160999000.0,146482000.0,1.324895e+08,157149732.6,1.558810e+08,1.532078e+08
4,41.200000,0.566670,271854000,287134000,308922000,190825000.0,200465000.0,215092000.0,81029500,86668900,...,3322050.0,3679470,27,326371333.0,326371333.0,308922000.0,2.894878e+08,314587132.6,3.427517e+08,3.253859e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,21.708333,112.261111,557032000,629419000,679747000,250848000.0,287891000.0,309439000.0,306184000,341528000,...,33703200.0,36056500,6,744780999.0,744781000.0,679747000.0,6.226765e+08,730542466.0,7.319428e+08,6.930402e+08
272,42.727839,-72.929108,123430000,131527000,145162000,103377000.0,110101000.0,121564000.0,20052600,21426000,...,3738910.0,4114260,33,155104999.0,155105000.0,145162000.0,1.334811e+08,154687199.4,1.579845e+08,1.540555e+08
273,35.411130,126.416190,649425000,710033000,750775000,404604000.0,440983000.0,464987000.0,244821000,269049000,...,6686990.0,6980200,17,804760999.0,804761000.0,750775000.0,7.039152e+08,821921466.0,8.299382e+08,7.813568e+08
274,47.511809,34.585460,273432000,274834000,272561000,162035000.0,163493000.0,162347000.0,111397000,111341000,...,6383160.0,5923510,31,272738000.0,272738000.0,272561000.0,2.736047e+08,281498999.8,3.244027e+08,3.068750e+08


In [31]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Assuming 'df' is your dataset

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Normalize each column in the dataset
for column in df.columns:
    # Reshape the data to 2D array before scaling
    column_data = df[column].values.reshape(-1, 1)
    df[column] = scaler.fit_transform(column_data)
df

,Latitude,Longitude,p90_1200,p00_1200,p10_1200,p90u_1200,p00u_1200,p10u_1200,p90r_1200,p00r_1200,...,p00r_300,p10r_300,Country_encoded,p20_1200LR,p20_1200LIB,P20_1200DT,P20_1200ridge,Predicted_p20_1200knn,ANN2,en
0,0.913304,0.489398,0.226290,0.187065,0.158115,0.249432,0.223774,0.210121,0.109688,0.090250,...,0.015467,0.013903,0.848485,0.137258,0.137258,0.158115,0.186554,0.134683,0.190114,0.184691
1,0.723163,0.407610,0.164938,0.146380,0.137234,0.180179,0.171884,0.178023,0.081266,0.073026,...,0.065309,0.062053,0.818182,0.127841,0.127841,0.137234,0.148104,0.127897,0.150417,0.147317
2,0.107433,0.274295,0.119590,0.112788,0.107343,0.139440,0.142382,0.150086,0.051805,0.048847,...,0.073693,0.070927,0.090909,0.103355,0.103355,0.107343,0.112735,0.092690,0.113736,0.112380
3,0.679075,0.106545,0.142137,0.131494,0.122977,0.190616,0.191717,0.198614,0.041441,0.037749,...,0.019304,0.018286,1.000000,0.116909,0.116909,0.122977,0.131297,0.111301,0.132749,0.130754
4,0.736806,0.429153,0.328496,0.284839,0.259634,0.392819,0.366816,0.369266,0.134377,0.117956,...,0.031856,0.030315,0.818182,0.237201,0.237201,0.259634,0.287269,0.248253,0.292289,0.285344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,0.545747,0.813314,0.673545,0.624773,0.571601,0.516392,0.526803,0.531251,0.509313,0.466063,...,0.326416,0.299600,0.181818,0.541551,0.541551,0.571601,0.618279,0.610088,0.624560,0.615443
272,0.751782,0.176372,0.148912,0.130300,0.121866,0.212784,0.201452,0.208687,0.032836,0.028842,...,0.035898,0.033932,1.000000,0.112622,0.112622,0.121866,0.132282,0.109159,0.134545,0.131515
273,0.680063,0.861999,0.785335,0.704834,0.631356,0.832940,0.806956,0.798312,0.407129,0.367065,...,0.064481,0.057768,0.515152,0.585180,0.585180,0.631356,0.698986,0.689577,0.708223,0.694738
274,0.798676,0.546157,0.330406,0.272623,0.229045,0.333547,0.299158,0.278708,0.184946,0.151655,...,0.061535,0.048980,0.939394,0.198188,0.198188,0.229045,0.271489,0.219471,0.276624,0.268724


In [32]:
from sklearn.decomposition import PCA

# Separate features and target variables
target_columns = ['p20_1200LR', 'p20_1200LIB', 'P20_1200DT', 'P20_1200ridge', 'Predicted_p20_1200knn', 'ANN2', 'en']
features = df.drop(target_columns, axis=1)
target_variables = df[target_columns]

# Apply dimensionality reduction using PCA
pca = PCA(n_components=5)  # Set the desired number of components
features_reduced = pca.fit_transform(features)

# Create a new DataFrame for the reduced features
df_reduced = pd.DataFrame(features_reduced, columns=['PC1', 'PC2', 'PC3', 'PC4', 'PC5'])

# Concatenate the reduced features with the target variables
df_combined = pd.concat([df_reduced, target_variables], axis=1)


In [33]:
df_combined

,PC1,PC2,PC3,PC4,PC5,p20_1200LR,p20_1200LIB,P20_1200DT,P20_1200ridge,Predicted_p20_1200knn,ANN2,en
0,-0.802412,0.070108,0.533996,-0.213804,-0.027186,0.137258,0.137258,0.158115,0.186554,0.134683,0.190114,0.184691
1,-0.809580,0.159435,-0.019536,0.127898,0.036270,0.127841,0.127841,0.137234,0.148104,0.127897,0.150417,0.147317
2,-0.034519,-0.280523,-0.363478,0.023808,0.644185,0.103355,0.103355,0.107343,0.112735,0.092690,0.113736,0.112380
3,-1.078497,0.039058,0.073401,0.177846,-0.135125,0.116909,0.116909,0.122977,0.131297,0.111301,0.132749,0.130754
4,-0.501050,0.171726,0.019281,0.036630,-0.029960,0.237201,0.237201,0.259634,0.287269,0.248253,0.292289,0.285344
...,...,...,...,...,...,...,...,...,...,...,...,...
271,1.232798,0.643078,-0.294524,0.178935,0.096370,0.541551,0.541551,0.571601,0.618279,0.610088,0.624560,0.615443
272,-0.369139,-0.368610,-0.332430,0.160723,-0.245654,0.112622,0.112622,0.121866,0.132282,0.109159,0.134545,0.131515
273,0.767564,0.617158,-0.073923,-0.267815,-0.203559,0.585180,0.585180,0.631356,0.698986,0.689577,0.708223,0.694738
274,-0.478618,0.140030,0.142593,0.061935,-0.085718,0.198188,0.198188,0.229045,0.271489,0.219471,0.276624,0.268724


In [34]:
# Define the number of folds for cross-validation
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True)
# Assuming you have multiple target variables named 'target_variable1', 'target_variable2', etc.
target_variables =  ['p20_1200LR', 'p20_1200LIB', 'P20_1200DT', 'P20_1200ridge', 'Predicted_p20_1200knn', 'ANN2', 'en']

In [74]:
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd

# Define the number of folds for cross-validation
n_splits = 5

# Define the test size
test_size = 0.3

# Create a DataFrame to store the errors
error_df = pd.DataFrame(columns=['Target Variable', 'RMSE', 'MAE', 'R2 Score', 'Explained Variance'])

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df_combined.drop(target_variables, axis=1),
                                                    df_combined[target_variables],
                                                    test_size=test_size,
                                                    random_state=42)

# Initialize the KFold
kf = KFold(n_splits=n_splits, shuffle=True)

# Define the parameter grid for GridSearchCV
param_grid = {
    'fit_intercept': [True, False],
}

# Iterate over the folds
for train_index, val_index in kf.split(X_train):
    # Get the training and validation sets
    X_train_fold, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val = y_train.iloc[train_index], y_train.iloc[val_index]

    # Initialize the linear regression model
    model = LinearRegression()

    # Perform GridSearchCV to find the best parameters
    grid_search = GridSearchCV(model, param_grid, cv=n_splits)
    grid_search.fit(X_train_fold, y_train_fold)

    # Get the best model with the best parameters



In [75]:
# Print the error DataFrame in a tabular format
print(error_df)

# Calculate the mean error
mean_rmse = error_df['RMSE'].mean()
mean_mae = error_df['MAE'].mean()
mean_r2 = error_df['R2 Score'].mean()
mean_explained_variance = error_df['Explained Variance'].mean()

print("Mean RMSE:", mean_rmse)
print("Mean MAE:", mean_mae)
print("Mean R2 Score:", mean_r2)
print("Mean Explained Variance:", mean_explained_variance)


Empty DataFrame
Columns: [Target Variable, RMSE, MAE, R2 Score, Explained Variance]
Index: []
Mean RMSE: nan
Mean MAE: nan
Mean R2 Score: nan
Mean Explained Variance: nan


In [76]:
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score
from sklearn.linear_model import Lasso
import numpy as np
import pandas as pd



# Define lists to store the errors for each target variable
lasso_rmse_list = []
lasso_mae_list = []
lasso_r2_list = []
lasso_explained_variance_list = []

# Create a DataFrame to store the errors
error_df = pd.DataFrame(columns=['Target Variable', 'RMSE', 'MAE', 'R2 Score', 'Explained Variance'])

# Define the parameter grid for GridSearchCV
param_grid = {
    'alpha': [0.1, 1.0, 10.0],  # Adjust the values of alpha as per your needs
    'max_iter': [100, 500, 1000]  # Adjust the values of max_iter as per your needs
}

# Iterate over the folds
for train_index, val_index in kf.split(X_train):
    # Get the training and validation sets
    X_train_fold, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val = y_train.iloc[train_index], y_train.iloc[val_index]

    # Initialize the Lasso regression model
    lasso_model = Lasso()

    # Perform GridSearchCV to find the best parameters
    grid_search = GridSearchCV(lasso_model, param_grid, cv=n_splits)
    grid_search.fit(X_train_fold, y_train_fold)

    # Get the best model with the best parameters
    best_lasso_model = grid_search.best_estimator_

    # Evaluate the model on the validation set
    y_pred_val = best_lasso_model.predict(X_val)

    # Calculate the evaluation metrics (e.g., RMSE, MAE, R2 score, explained variance score) for each target variable
    for i, target_variable in enumerate(target_variables):
        lasso_rmse_val = np.sqrt(mean_squared_error(y_val[target_variable], y_pred_val[:, i]))
        lasso_mae_val = mean_absolute_error(y_val[target_variable], y_pred_val[:, i])
        lasso_r2_val = r2_score(y_val[target_variable], y_pred_val[:, i])
        lasso_explained_variance_val = explained_variance_score(y_val[target_variable], y_pred_val[:, i])

        # Append the errors to the respective lists
        lasso_rmse_list.append(lasso_rmse_val)
        lasso_mae_list.append(lasso_mae_val)
        lasso_r2_list.append(lasso_r2_val)
        lasso_explained_variance_list.append(lasso_explained_variance_val)

        # Append the errors to the DataFrame
        error_df = error_df.append({
            'Target Variable': target_variable,
            'RMSE': lasso_rmse_val,
            'MAE': lasso_mae_val,
            'R2 Score': lasso_r2_val,
            'Explained Variance': lasso_explained_variance_val
        }, ignore_index=True)

        print("Lasso RMSE for", target_variable, ":", lasso_rmse_val)
        print("Lasso MAE for", target_variable, ":", lasso_mae_val)
        print("Lasso R2 score for", target_variable, ":", lasso_r2_val)
        print("Lasso Explained variance score for", target_variable, ":", lasso_explained_variance_val)

# Print the error DataFrame in a tabular format
print(error_df)

# Train the model with the best parameters on the full training set
best_lasso_model.fit(X_train, y_train)


<ipython-input-76-0f6a2dea899a>:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-76-0f6a2dea899a>:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-76-0f6a2dea899a>:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-76-0f6a2dea899a>:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-76-0f6a2dea899a>:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({


Lasso RMSE for p20_1200LR : 0.13197506393935432
Lasso MAE for p20_1200LR : 0.09631567122068684
Lasso R2 score for p20_1200LR : 0.19269694684333905
Lasso Explained variance score for p20_1200LR : 0.19283369481581314
Lasso RMSE for p20_1200LIB : 0.13197506394083702
Lasso MAE for p20_1200LIB : 0.09631567117956387
Lasso R2 score for p20_1200LIB : 0.19269694615647814
Lasso Explained variance score for p20_1200LIB : 0.192833694115489
Lasso RMSE for P20_1200DT : 0.13491544629209118
Lasso MAE for P20_1200DT : 0.10090217090922064
Lasso R2 score for P20_1200DT : 0.24648990800444648
Lasso Explained variance score for P20_1200DT : 0.24665305284359962
Lasso RMSE for P20_1200ridge : 0.14103476471521692
Lasso MAE for P20_1200ridge : 0.10825966948689977
Lasso R2 score for P20_1200ridge : 0.3071850875800438
Lasso Explained variance score for P20_1200ridge : 0.3072901629195104
Lasso RMSE for Predicted_p20_1200knn : 0.1477948547315394
Lasso MAE for Predicted_p20_1200knn : 0.10607438381751365
Lasso R2 sco

<ipython-input-76-0f6a2dea899a>:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-76-0f6a2dea899a>:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-76-0f6a2dea899a>:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-76-0f6a2dea899a>:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-76-0f6a2dea899a>:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({


Lasso RMSE for p20_1200LR : 0.18988929491421017
Lasso MAE for p20_1200LR : 0.12356245205576076
Lasso R2 score for p20_1200LR : -0.0036596388367471366
Lasso Explained variance score for p20_1200LR : 0.011112900396245573
Lasso RMSE for p20_1200LIB : 0.18988929481695693
Lasso MAE for p20_1200LIB : 0.12356245195254632
Lasso R2 score for p20_1200LIB : -0.0036596395107661994
Lasso Explained variance score for p20_1200LIB : 0.011112899789330277
Lasso RMSE for P20_1200DT : 0.1825460731372085
Lasso MAE for P20_1200DT : 0.12585014520991553
Lasso R2 score for P20_1200DT : 0.09930716520789507
Lasso Explained variance score for P20_1200DT : 0.11027236968366572
Lasso RMSE for P20_1200ridge : 0.17420686923147924
Lasso MAE for P20_1200ridge : 0.12997543311904872
Lasso R2 score for P20_1200ridge : 0.2284098169521923
Lasso Explained variance score for P20_1200ridge : 0.2349122809686578
Lasso RMSE for Predicted_p20_1200knn : 0.1776419403796494
Lasso MAE for Predicted_p20_1200knn : 0.1232851309803398
Lass

<ipython-input-76-0f6a2dea899a>:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-76-0f6a2dea899a>:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-76-0f6a2dea899a>:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-76-0f6a2dea899a>:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-76-0f6a2dea899a>:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({


Lasso RMSE for p20_1200LR : 0.09103531865149117
Lasso MAE for p20_1200LR : 0.07732092958062002
Lasso R2 score for p20_1200LR : 0.16719197440391376
Lasso Explained variance score for p20_1200LR : 0.2858630933149908
Lasso RMSE for p20_1200LIB : 0.09103531863237742
Lasso MAE for p20_1200LIB : 0.07732092955576135
Lasso R2 score for p20_1200LIB : 0.16719197330593816
Lasso Explained variance score for p20_1200LIB : 0.28586309286104683
Lasso RMSE for P20_1200DT : 0.09681942158171547
Lasso MAE for P20_1200DT : 0.08314352701252235
Lasso R2 score for P20_1200DT : 0.23015706477878128
Lasso Explained variance score for P20_1200DT : 0.3201165223966346
Lasso RMSE for P20_1200ridge : 0.10610422257794215
Lasso MAE for P20_1200ridge : 0.09146585514205774
Lasso R2 score for P20_1200ridge : 0.29043061622743194
Lasso Explained variance score for P20_1200ridge : 0.35308520721059355
Lasso RMSE for Predicted_p20_1200knn : 0.10229599356643289
Lasso MAE for Predicted_p20_1200knn : 0.08585089959672834
Lasso R2 

<ipython-input-76-0f6a2dea899a>:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-76-0f6a2dea899a>:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-76-0f6a2dea899a>:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-76-0f6a2dea899a>:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-76-0f6a2dea899a>:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({


Lasso RMSE for p20_1200LR : 0.13702806464270714
Lasso MAE for p20_1200LR : 0.10659909628481962
Lasso R2 score for p20_1200LR : 0.17785981084941005
Lasso Explained variance score for p20_1200LR : 0.17933209160021002
Lasso RMSE for p20_1200LIB : 0.1370280646329162
Lasso MAE for p20_1200LIB : 0.10659909632355154
Lasso R2 score for p20_1200LIB : 0.17785981014125496
Lasso Explained variance score for p20_1200LIB : 0.17933209084970314
Lasso RMSE for P20_1200DT : 0.13848097914666146
Lasso MAE for P20_1200DT : 0.11057244600844515
Lasso R2 score for P20_1200DT : 0.23916671223086572
Lasso Explained variance score for P20_1200DT : 0.2399269050718995
Lasso RMSE for P20_1200ridge : 0.14269865382394198
Lasso MAE for P20_1200ridge : 0.11649913740149663
Lasso R2 score for P20_1200ridge : 0.30930141795662536
Lasso Explained variance score for P20_1200ridge : 0.30953582719032746
Lasso RMSE for Predicted_p20_1200knn : 0.1446985625300977
Lasso MAE for Predicted_p20_1200knn : 0.11148925487375745
Lasso R2 s

<ipython-input-76-0f6a2dea899a>:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-76-0f6a2dea899a>:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-76-0f6a2dea899a>:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-76-0f6a2dea899a>:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-76-0f6a2dea899a>:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({


Lasso(alpha=0.1, max_iter=100)

In [77]:
# Print the error DataFrame in a tabular format
print(error_df)

# Calculate the mean error
mean_lasso_rmse = np.mean(lasso_rmse_list)
mean_lasso_mae = np.mean(lasso_mae_list)
mean_lasso_r2 = np.mean(lasso_r2_list)
mean_lasso_explained_variance = np.mean(lasso_explained_variance_list)

print("Mean Lasso RMSE:", mean_lasso_rmse)
print("Mean Lasso MAE:", mean_lasso_mae)
print("Mean Lasso R2 Score:", mean_lasso_r2)
print("Mean Lasso Explained Variance:", mean_lasso_explained_variance)



          Target Variable      RMSE       MAE  R2 Score  Explained Variance
0              p20_1200LR  0.131975  0.096316  0.192697            0.192834
1             p20_1200LIB  0.131975  0.096316  0.192697            0.192834
2              P20_1200DT  0.134915  0.100902  0.246490            0.246653
3           P20_1200ridge  0.141035  0.108260  0.307185            0.307290
4   Predicted_p20_1200knn  0.147795  0.106074  0.272525            0.272553
5                    ANN2  0.141883  0.109254  0.314516            0.314633
6                      en  0.140633  0.107778  0.303753            0.303858
7              p20_1200LR  0.189889  0.123562 -0.003660            0.011113
8             p20_1200LIB  0.189889  0.123562 -0.003660            0.011113
9              P20_1200DT  0.182546  0.125850  0.099307            0.110272
10          P20_1200ridge  0.174207  0.129975  0.228410            0.234912
11  Predicted_p20_1200knn  0.177642  0.123285  0.197118            0.203925
12          

In [78]:
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score
from sklearn.linear_model import Ridge
import numpy as np
import pandas as pd



# Define lists to store the errors for each target variable
ridge_rmse_list = []
ridge_mae_list = []
ridge_r2_list = []
ridge_explained_variance_list = []

# Create a DataFrame to store the errors
error_df = pd.DataFrame(columns=['Target Variable', 'RMSE', 'MAE', 'R2 Score', 'Explained Variance'])

# Define the parameter grid for GridSearchCV
ridge_param_grid = {
    'alpha': [0.1, 1.0, 10.0],  # Adjust the values of alpha as per your needs
    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],  # Adjust the solver options as per your needs
    'max_iter': [1000, 2000, 5000]  # Include additional parameter options
}

# Iterate over the folds
for train_index, val_index in kf.split(X_train):
    # Get the training and validation sets
    X_train_fold, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val = y_train.iloc[train_index], y_train.iloc[val_index]

    # Initialize the Ridge regression model
    ridge_model = Ridge()

    # Perform GridSearchCV to find the best parameters
    grid_search = GridSearchCV(ridge_model, ridge_param_grid, cv=n_splits)
    grid_search.fit(X_train_fold, y_train_fold)

    # Get the best model with the best parameters
    best_ridge_model = grid_search.best_estimator_

    # Evaluate the model on the validation set
    y_pred_val = best_ridge_model.predict(X_val)

    # Calculate the evaluation metrics (e.g., RMSE, MAE, R2 score, explained variance score) for each target variable
    for i, target_variable in enumerate(target_variables):
        ridge_rmse_val = np.sqrt(mean_squared_error(y_val[target_variable], y_pred_val[:, i]))
        ridge_mae_val = mean_absolute_error(y_val[target_variable], y_pred_val[:, i])
        ridge_r2_val = r2_score(y_val[target_variable], y_pred_val[:, i])
        ridge_explained_variance_val = explained_variance_score(y_val[target_variable], y_pred_val[:, i])

        # Append the errors to the respective lists
        ridge_rmse_list.append(ridge_rmse_val)
        ridge_mae_list.append(ridge_mae_val)
        ridge_r2_list.append(ridge_r2_val)
        ridge_explained_variance_list.append(ridge_explained_variance_val)

        # Append the errors to the DataFrame
        error_df = error_df.append({
            'Target Variable': target_variable,
            'RMSE': ridge_rmse_val,
            'MAE': ridge_mae_val,
            'R2 Score': ridge_r2_val,
            'Explained Variance': ridge_explained_variance_val
        }, ignore_index=True)

        print("Ridge RMSE for", target_variable, ":", ridge_rmse_val)
        print("Ridge MAE for", target_variable, ":", ridge_mae_val)
        print("Ridge R2 score for", target_variable, ":", ridge_r2_val)
        print("Ridge Explained variance score for", target_variable, ":", ridge_explained_variance_val)

# Print the error DataFrame in a tabular format
print(error_df)

# Train the model with the best parameters on the full training set
best_ridge_model.fit(X_train, y_train)



<ipython-input-78-a005ba1cfbf5>:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-78-a005ba1cfbf5>:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-78-a005ba1cfbf5>:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-78-a005ba1cfbf5>:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-78-a005ba1cfbf5>:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({


Ridge RMSE for p20_1200LR : 0.03264916195802955
Ridge MAE for p20_1200LR : 0.025890991052265975
Ridge R2 score for p20_1200LR : 0.9533218327125378
Ridge Explained variance score for p20_1200LR : 0.9572824108538438
Ridge RMSE for p20_1200LIB : 0.032662492480565804
Ridge MAE for p20_1200LIB : 0.025904102661067376
Ridge R2 score for p20_1200LIB : 0.9532837078150352
Ridge Explained variance score for p20_1200LIB : 0.9572471512132428
Ridge RMSE for P20_1200DT : 0.030777313531047103
Ridge MAE for P20_1200DT : 0.02319103539696729
Ridge R2 score for P20_1200DT : 0.9607301010155777
Ridge Explained variance score for P20_1200DT : 0.9622962921631433
Ridge RMSE for P20_1200ridge : 0.03133963848570503
Ridge MAE for P20_1200ridge : 0.02339804426906487
Ridge R2 score for P20_1200ridge : 0.9627020780891442
Ridge Explained variance score for P20_1200ridge : 0.9628779355255463
Ridge RMSE for Predicted_p20_1200knn : 0.03920096443544719
Ridge MAE for Predicted_p20_1200knn : 0.02980616758929167
Ridge R2 sc

<ipython-input-78-a005ba1cfbf5>:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-78-a005ba1cfbf5>:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-78-a005ba1cfbf5>:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-78-a005ba1cfbf5>:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-78-a005ba1cfbf5>:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({


Ridge RMSE for p20_1200LR : 0.04352723560658181
Ridge MAE for p20_1200LR : 0.02915919477809534
Ridge R2 score for p20_1200LR : 0.9390197217161333
Ridge Explained variance score for p20_1200LR : 0.9406006300412758
Ridge RMSE for p20_1200LIB : 0.04352502255093475
Ridge MAE for p20_1200LIB : 0.029158543648315424
Ridge R2 score for p20_1200LIB : 0.9390259222946091
Ridge Explained variance score for p20_1200LIB : 0.9406067440968016
Ridge RMSE for P20_1200DT : 0.039662596056554755
Ridge MAE for P20_1200DT : 0.027432670074513343
Ridge R2 score for P20_1200DT : 0.951847164818922
Ridge Explained variance score for P20_1200DT : 0.9528344446413674
Ridge RMSE for P20_1200ridge : 0.03884658413733169
Ridge MAE for P20_1200ridge : 0.029677284919424222
Ridge R2 score for P20_1200ridge : 0.9584032492886464
Ridge Explained variance score for P20_1200ridge : 0.959013657440864
Ridge RMSE for Predicted_p20_1200knn : 0.04558685141598199
Ridge MAE for Predicted_p20_1200knn : 0.0320701667191525
Ridge R2 score

<ipython-input-78-a005ba1cfbf5>:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-78-a005ba1cfbf5>:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-78-a005ba1cfbf5>:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-78-a005ba1cfbf5>:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-78-a005ba1cfbf5>:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({


Ridge RMSE for p20_1200LR : 0.04347151425171859
Ridge MAE for p20_1200LR : 0.02751271483196309
Ridge R2 score for p20_1200LR : 0.9346496987516727
Ridge Explained variance score for p20_1200LR : 0.9420079037050173
Ridge RMSE for p20_1200LIB : 0.04346850898757687
Ridge MAE for p20_1200LIB : 0.02751209066930537
Ridge R2 score for p20_1200LIB : 0.9346587338958072
Ridge Explained variance score for p20_1200LIB : 0.9420155502381726
Ridge RMSE for P20_1200DT : 0.04097367540084022
Ridge MAE for P20_1200DT : 0.025006111015567052
Ridge R2 score for P20_1200DT : 0.948732831603531
Ridge Explained variance score for P20_1200DT : 0.9529603723179771
Ridge RMSE for P20_1200ridge : 0.042262130260566244
Ridge MAE for P20_1200ridge : 0.025871671861359892
Ridge R2 score for P20_1200ridge : 0.9549398376846833
Ridge Explained variance score for P20_1200ridge : 0.956606350370762
Ridge RMSE for Predicted_p20_1200knn : 0.05404000099748466
Ridge MAE for Predicted_p20_1200knn : 0.03186604863410696
Ridge R2 score

<ipython-input-78-a005ba1cfbf5>:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-78-a005ba1cfbf5>:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-78-a005ba1cfbf5>:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-78-a005ba1cfbf5>:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-78-a005ba1cfbf5>:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({


Ridge RMSE for p20_1200LR : 0.031476420336676025
Ridge MAE for p20_1200LR : 0.024191317601569996
Ridge R2 score for p20_1200LR : 0.951167510516225
Ridge Explained variance score for p20_1200LR : 0.9537749757340571
Ridge RMSE for p20_1200LIB : 0.031476519573666914
Ridge MAE for p20_1200LIB : 0.024191291174986255
Ridge R2 score for p20_1200LIB : 0.9511672025768191
Ridge Explained variance score for p20_1200LIB : 0.9537749247795534
Ridge RMSE for P20_1200DT : 0.031311103222333894
Ridge MAE for P20_1200DT : 0.023356673363232078
Ridge R2 score for P20_1200DT : 0.9574613898679525
Ridge Explained variance score for P20_1200DT : 0.9596726332981905
Ridge RMSE for P20_1200ridge : 0.03411412608621486
Ridge MAE for P20_1200ridge : 0.02553892044177881
Ridge R2 score for P20_1200ridge : 0.9577766563876092
Ridge Explained variance score for P20_1200ridge : 0.95947628054731
Ridge RMSE for Predicted_p20_1200knn : 0.03820062996984202
Ridge MAE for Predicted_p20_1200knn : 0.028065442044157722
Ridge R2 sc

<ipython-input-78-a005ba1cfbf5>:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-78-a005ba1cfbf5>:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-78-a005ba1cfbf5>:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-78-a005ba1cfbf5>:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-78-a005ba1cfbf5>:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({


Ridge(alpha=0.1, max_iter=1000, solver='sparse_cg')

In [79]:
# Calculate the mean error for Ridge regression
mean_ridge_rmse = np.mean(ridge_rmse_list)
mean_ridge_mae = np.mean(ridge_mae_list)
mean_ridge_r2 = np.mean(ridge_r2_list)
mean_ridge_explained_variance = np.mean(ridge_explained_variance_list)

print("Mean Ridge RMSE:", mean_ridge_rmse)
print("Mean Ridge MAE:", mean_ridge_mae)
print("Mean Ridge R2 Score:", mean_ridge_r2)
print("Mean Ridge Explained Variance:", mean_ridge_explained_variance)


Mean Ridge RMSE: 0.036731311020864285
Mean Ridge MAE: 0.025941137786495778
Mean Ridge R2 Score: 0.9394214376599752
Mean Ridge Explained Variance: 0.9418609395747837


In [80]:
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score
from sklearn.neighbors import KNeighborsRegressor
import numpy as np
import pandas as pd



# Define lists to store the errors for each target variable
knn_rmse_list = []
knn_mae_list = []
knn_r2_list = []
knn_explained_variance_list = []

# Create a DataFrame to store the errors
error_df = pd.DataFrame(columns=['Target Variable', 'RMSE', 'MAE', 'R2 Score', 'Explained Variance'])

# Define the parameter grid for GridSearchCV
knn_param_grid = {
    'n_neighbors': [3, 5, 7],  # Adjust the number of neighbors as per your needs
    'weights': ['uniform', 'distance'],  # Adjust the weight options as per your needs
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],  # Adjust the algorithm options as per your needs
    'leaf_size': [10, 30, 50],  # Adjust the leaf size options as per your needs
    'p': [1, 2],  # Adjust the distance metric options as per your needs
}

# Iterate over the folds
for train_index, val_index in kf.split(X_train):
    # Get the training and validation sets
    X_train_fold, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val = y_train.iloc[train_index], y_train.iloc[val_index]

    # Initialize the KNN regression model
    knn_model = KNeighborsRegressor()

    # Perform GridSearchCV to find the best parameters
    grid_search = GridSearchCV(knn_model, knn_param_grid, cv=n_splits)
    grid_search.fit(X_train_fold, y_train_fold)

    # Get the best model with the best parameters
    best_knn_model = grid_search.best_estimator_

    # Evaluate the model on the validation set
    y_pred_val = best_knn_model.predict(X_val)

    # Calculate the evaluation metrics (e.g., RMSE, MAE, R2 score, explained variance score) for each target variable
    for i, target_variable in enumerate(target_variables):
        knn_rmse_val = np.sqrt(mean_squared_error(y_val[target_variable], y_pred_val[:, i]))
        knn_mae_val = mean_absolute_error(y_val[target_variable], y_pred_val[:, i])
        knn_r2_val = r2_score(y_val[target_variable], y_pred_val[:, i])
        knn_explained_variance_val = explained_variance_score(y_val[target_variable], y_pred_val[:, i])

        # Append the errors to the respective lists
        knn_rmse_list.append(knn_rmse_val)
        knn_mae_list.append(knn_mae_val)
        knn_r2_list.append(knn_r2_val)
        knn_explained_variance_list.append(knn_explained_variance_val)

        # Append the errors to the DataFrame
        error_df = error_df.append({
            'Target Variable': target_variable,
            'RMSE': knn_rmse_val,
            'MAE': knn_mae_val,
            'R2 Score': knn_r2_val,
            'Explained Variance': knn_explained_variance_val
        }, ignore_index=True)

        print("KNN RMSE for", target_variable, ":", knn_rmse_val)
        print("KNN MAE for", target_variable, ":", knn_mae_val)
        print("KNN R2 score for", target_variable, ":", knn_r2_val)
        print("KNN Explained variance score for", target_variable, ":", knn_explained_variance_val)

# Print the error DataFrame in a tabular format
print(error_df)




<ipython-input-80-cee152b4504a>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-80-cee152b4504a>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-80-cee152b4504a>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-80-cee152b4504a>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-80-cee152b4504a>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({


KNN RMSE for p20_1200LR : 0.04452812295602301
KNN MAE for p20_1200LR : 0.028522364392582728
KNN R2 score for p20_1200LR : 0.9207548285941769
KNN Explained variance score for p20_1200LR : 0.9212531965653667
KNN RMSE for p20_1200LIB : 0.044528122875234784
KNN MAE for p20_1200LIB : 0.028522364324754097
KNN R2 score for p20_1200LIB : 0.9207548287311954
KNN Explained variance score for p20_1200LIB : 0.9212531967002954
KNN RMSE for P20_1200DT : 0.04809303414334291
KNN MAE for P20_1200DT : 0.030837688326615955
KNN R2 score for P20_1200DT : 0.9184296847799581
KNN Explained variance score for P20_1200DT : 0.9189517445447758
KNN RMSE for P20_1200ridge : 0.053940546337806844
KNN MAE for P20_1200ridge : 0.03502383741569625
KNN R2 score for P20_1200ridge : 0.9145167595741764
KNN Explained variance score for P20_1200ridge : 0.9150927825191184
KNN RMSE for Predicted_p20_1200knn : 0.05445456416592613
KNN MAE for Predicted_p20_1200knn : 0.03455049979100288
KNN R2 score for Predicted_p20_1200knn : 0.915

<ipython-input-80-cee152b4504a>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-80-cee152b4504a>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-80-cee152b4504a>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-80-cee152b4504a>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-80-cee152b4504a>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({


KNN RMSE for p20_1200LR : 0.06629304495076103
KNN MAE for p20_1200LR : 0.03557713149896577
KNN R2 score for p20_1200LR : 0.8521454734096126
KNN Explained variance score for p20_1200LR : 0.8626497351581366
KNN RMSE for p20_1200LIB : 0.06629304505324357
KNN MAE for p20_1200LIB : 0.03557713155262988
KNN R2 score for p20_1200LIB : 0.8521454729835657
KNN Explained variance score for p20_1200LIB : 0.8626497346941399
KNN RMSE for P20_1200DT : 0.0661643706989589
KNN MAE for P20_1200DT : 0.036182115265560436
KNN R2 score for P20_1200DT : 0.8652275564181366
KNN Explained variance score for P20_1200DT : 0.8750640768684077
KNN RMSE for P20_1200ridge : 0.06733022955423304
KNN MAE for P20_1200ridge : 0.03784373285571085
KNN R2 score for P20_1200ridge : 0.8791161679705984
KNN Explained variance score for P20_1200ridge : 0.8886222739855345
KNN RMSE for Predicted_p20_1200knn : 0.07268256804913688
KNN MAE for Predicted_p20_1200knn : 0.03835138027605341
KNN R2 score for Predicted_p20_1200knn : 0.86027317

<ipython-input-80-cee152b4504a>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-80-cee152b4504a>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-80-cee152b4504a>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-80-cee152b4504a>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-80-cee152b4504a>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({


KNN RMSE for p20_1200LR : 0.030267434050375547
KNN MAE for p20_1200LR : 0.021255209154151754
KNN R2 score for p20_1200LR : 0.9512165911008699
KNN Explained variance score for p20_1200LR : 0.9534145581830704
KNN RMSE for p20_1200LIB : 0.030267434123374504
KNN MAE for p20_1200LIB : 0.0212552091564158
KNN R2 score for p20_1200LIB : 0.9512165908327698
KNN Explained variance score for p20_1200LIB : 0.9534145579358354
KNN RMSE for P20_1200DT : 0.03233795056427661
KNN MAE for P20_1200DT : 0.022345198089330408
KNN R2 score for P20_1200DT : 0.9493922474849219
KNN Explained variance score for P20_1200DT : 0.9539888473405725
KNN RMSE for P20_1200ridge : 0.03830969702859179
KNN MAE for P20_1200ridge : 0.02695708203222819
KNN R2 score for P20_1200ridge : 0.9394246385880496
KNN Explained variance score for P20_1200ridge : 0.9479436807303675
KNN RMSE for Predicted_p20_1200knn : 0.03339540297352185
KNN MAE for Predicted_p20_1200knn : 0.022897880080603533
KNN R2 score for Predicted_p20_1200knn : 0.9532

<ipython-input-80-cee152b4504a>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-80-cee152b4504a>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-80-cee152b4504a>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-80-cee152b4504a>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-80-cee152b4504a>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({


KNN RMSE for p20_1200LR : 0.025388799626743095
KNN MAE for p20_1200LR : 0.014374277499960385
KNN R2 score for p20_1200LR : 0.9331437907136013
KNN Explained variance score for p20_1200LR : 0.9338111838219026
KNN RMSE for p20_1200LIB : 0.025388799547518832
KNN MAE for p20_1200LIB : 0.014374277391589129
KNN R2 score for p20_1200LIB : 0.9331437909670195
KNN Explained variance score for p20_1200LIB : 0.9338111840381044
KNN RMSE for P20_1200DT : 0.026777545568370528
KNN MAE for P20_1200DT : 0.015930573263600787
KNN R2 score for P20_1200DT : 0.9410631239537
KNN Explained variance score for P20_1200DT : 0.9417133090632774
KNN RMSE for P20_1200ridge : 0.029768177770749693
KNN MAE for P20_1200ridge : 0.018365255395266077
KNN R2 score for P20_1200ridge : 0.9450339116846702
KNN Explained variance score for P20_1200ridge : 0.9456287608139144
KNN RMSE for Predicted_p20_1200knn : 0.03165109330963782
KNN MAE for Predicted_p20_1200knn : 0.018136418198623355
KNN R2 score for Predicted_p20_1200knn : 0.92

<ipython-input-80-cee152b4504a>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-80-cee152b4504a>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-80-cee152b4504a>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-80-cee152b4504a>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-80-cee152b4504a>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({


In [81]:
# Calculate the mean error for KNN Regression
knn_mean_rmse = np.mean(knn_rmse_list)
knn_mean_mae = np.mean(knn_mae_list)
knn_mean_r2 = np.mean(knn_r2_list)
knn_mean_explained_variance = np.mean(knn_explained_variance_list)

print("\nMean KNN RMSE:", knn_mean_rmse)
print("Mean KNN MAE:", knn_mean_mae)
print("Mean KNN R2 Score:", knn_mean_r2)
print("Mean KNN Explained Variance:", knn_mean_explained_variance)


Mean KNN RMSE: 0.04542115494777294
Mean KNN MAE: 0.026756564873186152
Mean KNN R2 Score: 0.9217350057934364
Mean KNN Explained Variance: 0.9256835451838727


In [82]:
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score
from sklearn.neural_network import MLPRegressor
import numpy as np
import pandas as pd


# Define lists to store the errors for each target variable
ann_rmse_list = []
ann_mae_list = []
ann_r2_list = []
ann_explained_variance_list = []

# Create a DataFrame to store the errors
error_df = pd.DataFrame(columns=['Target Variable', 'RMSE', 'MAE', 'R2 Score', 'Explained Variance'])

# Define the parameter grid for GridSearchCV
ann_param_grid = {
    'hidden_layer_sizes': [(100,), (50, 50), (50, 50, 50)],  # Adjust the hidden layer sizes as per your needs
    'activation': ['relu', 'tanh'],  # Adjust the activation function options as per your needs
    'solver': ['adam', 'sgd'],  # Adjust the solver options as per your needs
    'learning_rate': ['constant', 'adaptive'],  # Adjust the learning rate options as per your needs
    'alpha': [0.0001],  # Adjust the regularization parameter as per your needs
    'batch_size': ['auto'],  # Adjust the batch size as per your needs
    'learning_rate_init': [0.001],  # Adjust the initial learning rate as per your needs
    'power_t': [0.5],  # Adjust the power parameter as per your needs
    'max_iter': [200],  # Adjust the maximum number of iterations as per your needs
    'shuffle': [True],  # Adjust the shuffle parameter as per your needs
    'random_state': [None],  # Adjust the random state as per your needs
    'tol': [0.0001],  # Adjust the tolerance as per your needs
    'verbose': [False],  # Adjust the verbosity level as per your needs
    'warm_start': [False],  # Adjust the warm start parameter as per your needs
    'momentum': [0.9],  # Adjust the momentum parameter as per your needs
    'nesterovs_momentum': [True],  # Adjust the Nesterov's momentum parameter as per your needs
    'early_stopping': [False],  # Adjust the early stopping parameter as per your needs
    'validation_fraction': [0.1],  # Adjust the validation fraction as per your needs
    'beta_1': [0.9],  # Adjust the beta 1 parameter as per your needs
    'beta_2': [0.999],  # Adjust the beta 2 parameter as per your needs
    'epsilon': [1e-08],  # Adjust the epsilon parameter as per your needs
    'n_iter_no_change': [10],  # Adjust the number of iterations with no improvement as per your needs
    'max_fun': [15000]  # Adjust the maximum number of function evaluations as per your needs
}

# Iterate over the folds
for train_index, val_index in kf.split(X_train):
    # Get the training and validation sets
    X_train_fold, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val = y_train.iloc[train_index], y_train.iloc[val_index]

    # Initialize the ANN regression model
    ann_model = MLPRegressor()

    # Perform GridSearchCV to find the best parameters
    grid_search = GridSearchCV(ann_model, ann_param_grid, cv=n_splits)
    grid_search.fit(X_train_fold, y_train_fold)

    # Get the best model with the best parameters
    best_ann_model = grid_search.best_estimator_

    # Fit the model on the training fold
    best_ann_model.fit(X_train_fold, y_train_fold)

    # Evaluate the model on the validation set
    y_pred_val = best_ann_model.predict(X_val)

    # Calculate the evaluation metrics (e.g., RMSE, MAE, R2 score, explained variance score) for each target variable
    for i, target_variable in enumerate(target_variables):
        ann_rmse_val = np.sqrt(mean_squared_error(y_val[target_variable], y_pred_val[:, i]))
        ann_mae_val = mean_absolute_error(y_val[target_variable], y_pred_val[:, i])
        ann_r2_val = r2_score(y_val[target_variable], y_pred_val[:, i])
        ann_explained_variance_val = explained_variance_score(y_val[target_variable], y_pred_val[:, i])

        # Append the errors to the respective lists
        ann_rmse_list.append(ann_rmse_val)
        ann_mae_list.append(ann_mae_val)
        ann_r2_list.append(ann_r2_val)
        ann_explained_variance_list.append(ann_explained_variance_val)

        # Append the errors to the DataFrame
        error_df = error_df.append({
            'Target Variable': target_variable,
            'RMSE': ann_rmse_val,
            'MAE': ann_mae_val,
            'R2 Score': ann_r2_val,
            'Explained Variance': ann_explained_variance_val
        }, ignore_index=True)

        print("ANN RMSE for", target_variable, ":", ann_rmse_val)
        print("ANN MAE for", target_variable, ":", ann_mae_val)
        print("ANN R2 score for", target_variable, ":", ann_r2_val)
        print("ANN Explained variance score for", target_variable, ":", ann_explained_variance_val)

# Print the error DataFrame in a tabular format
print(error_df)


<ipython-input-82-94e793a5a7c9>:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-82-94e793a5a7c9>:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-82-94e793a5a7c9>:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-82-94e793a5a7c9>:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-82-94e793a5a7c9>:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({


ANN RMSE for p20_1200LR : 0.07174105597539206
ANN MAE for p20_1200LR : 0.05596734494029912
ANN R2 score for p20_1200LR : 0.794516291526169
ANN Explained variance score for p20_1200LR : 0.8162703574875139
ANN RMSE for p20_1200LIB : 0.047444081352186654
ANN MAE for p20_1200LIB : 0.03418385168498944
ANN R2 score for p20_1200LIB : 0.9101318194279505
ANN Explained variance score for p20_1200LIB : 0.9157992014260872
ANN RMSE for P20_1200DT : 0.04818728891834617
ANN MAE for P20_1200DT : 0.030651769193952554
ANN R2 score for P20_1200DT : 0.9139992655991672
ANN Explained variance score for P20_1200DT : 0.9171905843775449
ANN RMSE for P20_1200ridge : 0.04582082550743092
ANN MAE for P20_1200ridge : 0.036597525603144415
ANN R2 score for P20_1200ridge : 0.931326280378592
ANN Explained variance score for P20_1200ridge : 0.9400025144222557
ANN RMSE for Predicted_p20_1200knn : 0.06262253671943153
ANN MAE for Predicted_p20_1200knn : 0.04907219343788064
ANN R2 score for Predicted_p20_1200knn : 0.8696157

<ipython-input-82-94e793a5a7c9>:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-82-94e793a5a7c9>:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-82-94e793a5a7c9>:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-82-94e793a5a7c9>:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-82-94e793a5a7c9>:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({


ANN RMSE for p20_1200LR : 0.04273687562564857
ANN MAE for p20_1200LR : 0.03299317378853187
ANN R2 score for p20_1200LR : 0.8831963730999726
ANN Explained variance score for p20_1200LR : 0.9081687598675241
ANN RMSE for p20_1200LIB : 0.039524305438543385
ANN MAE for p20_1200LIB : 0.031087762423739524
ANN R2 score for p20_1200LIB : 0.900096826342674
ANN Explained variance score for p20_1200LIB : 0.9169917252111276
ANN RMSE for P20_1200DT : 0.03351216623896151
ANN MAE for P20_1200DT : 0.027548026777027015
ANN R2 score for P20_1200DT : 0.9335772425452504
ANN Explained variance score for P20_1200DT : 0.9494429508128976
ANN RMSE for P20_1200ridge : 0.04248935862875548
ANN MAE for P20_1200ridge : 0.02673995099115203
ANN R2 score for P20_1200ridge : 0.9079000947307737
ANN Explained variance score for P20_1200ridge : 0.9162618736187101
ANN RMSE for Predicted_p20_1200knn : 0.03498254611867555
ANN MAE for Predicted_p20_1200knn : 0.02850850887579023
ANN R2 score for Predicted_p20_1200knn : 0.938794

<ipython-input-82-94e793a5a7c9>:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-82-94e793a5a7c9>:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-82-94e793a5a7c9>:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-82-94e793a5a7c9>:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-82-94e793a5a7c9>:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({


ANN RMSE for p20_1200LR : 0.03925785238247494
ANN MAE for p20_1200LR : 0.02725043109121131
ANN R2 score for p20_1200LR : 0.8889146661744619
ANN Explained variance score for p20_1200LR : 0.8902235808950876
ANN RMSE for p20_1200LIB : 0.04211387010750255
ANN MAE for p20_1200LIB : 0.02823874806035996
ANN R2 score for p20_1200LIB : 0.8721637695824419
ANN Explained variance score for p20_1200LIB : 0.874231291117714
ANN RMSE for P20_1200DT : 0.05043443504659665
ANN MAE for P20_1200DT : 0.035064199090068804
ANN R2 score for P20_1200DT : 0.8531949483469087
ANN Explained variance score for P20_1200DT : 0.8641124006591678
ANN RMSE for P20_1200ridge : 0.04312516231786366
ANN MAE for P20_1200ridge : 0.03088333015342186
ANN R2 score for P20_1200ridge : 0.9182985913566786
ANN Explained variance score for P20_1200ridge : 0.9189284830071157
ANN RMSE for Predicted_p20_1200knn : 0.05172208825625985
ANN MAE for Predicted_p20_1200knn : 0.034414446272982034
ANN R2 score for Predicted_p20_1200knn : 0.8670515

<ipython-input-82-94e793a5a7c9>:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-82-94e793a5a7c9>:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-82-94e793a5a7c9>:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-82-94e793a5a7c9>:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-82-94e793a5a7c9>:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({


ANN RMSE for p20_1200LR : 0.04478361388427016
ANN MAE for p20_1200LR : 0.03372700877730572
ANN R2 score for p20_1200LR : 0.9483208551386113
ANN Explained variance score for p20_1200LR : 0.9489897763757998
ANN RMSE for p20_1200LIB : 0.050806687716346784
ANN MAE for p20_1200LIB : 0.03735813529417474
ANN R2 score for p20_1200LIB : 0.9334851201168555
ANN Explained variance score for p20_1200LIB : 0.9339553566212357
ANN RMSE for P20_1200DT : 0.050383151953412225
ANN MAE for P20_1200DT : 0.03974991048555189
ANN R2 score for P20_1200DT : 0.9384244212700524
ANN Explained variance score for P20_1200DT : 0.958770775147834
ANN RMSE for P20_1200ridge : 0.04534533451441904
ANN MAE for P20_1200ridge : 0.031835408580773154
ANN R2 score for P20_1200ridge : 0.9551699931796429
ANN Explained variance score for P20_1200ridge : 0.955411257732382
ANN RMSE for Predicted_p20_1200knn : 0.0618461306688882
ANN MAE for Predicted_p20_1200knn : 0.04709323721416274
ANN R2 score for Predicted_p20_1200knn : 0.91982501

<ipython-input-82-94e793a5a7c9>:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-82-94e793a5a7c9>:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-82-94e793a5a7c9>:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-82-94e793a5a7c9>:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-82-94e793a5a7c9>:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({


In [83]:
# Calculate the mean error for ANN Regression
ann_mean_rmse = np.mean(ann_rmse_list)
ann_mean_mae = np.mean(ann_mae_list)
ann_mean_r2 = np.mean(ann_r2_list)
ann_mean_explained_variance = np.mean(ann_explained_variance_list)

print("\nMean ANN RMSE:", ann_mean_rmse)
print("Mean ANN MAE:", ann_mean_mae)
print("Mean ANN R2 Score:", ann_mean_r2)
print("Mean ANN Explained Variance:", ann_mean_explained_variance)





Mean ANN RMSE: 0.04772487040395881
Mean ANN MAE: 0.03522078810957603
Mean ANN R2 Score: 0.9055942622028369
Mean ANN Explained Variance: 0.9213065469450499


In [84]:
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score
from sklearn.linear_model import ElasticNet
import numpy as np
import pandas as pd


# Define lists to store the errors for each target variable
en_rmse_list = []
en_mae_list = []
en_r2_list = []
en_explained_variance_list = []

# Create a DataFrame to store the errors
error_df = pd.DataFrame(columns=['Target Variable', 'RMSE', 'MAE', 'R2 Score', 'Explained Variance'])

# Define the parameter grid for GridSearchCV
en_param_grid = {
    'alpha': [0.1, 1.0, 10.0],  # Adjust the values of alpha as per your needs
    'l1_ratio': [0.2, 0.5, 0.8],  # Adjust the values of l1_ratio as per your needs
    'fit_intercept': [True, False],  # Adjust the fit_intercept option as per your needs
    'precompute': [True, False],  # Adjust the precompute option as per your needs
    'max_iter': [1000],  # Adjust the maximum number of iterations as per your needs
    'tol': [0.0001],  # Adjust the tolerance as per your needs
    'warm_start': [True, False],  # Adjust the warm_start option as per your needs
    'positive': [False],  # Adjust the positive option as per your needs
    'random_state': [None],  # Adjust the random_state as per your needs
    'selection': ['cyclic', 'random']  # Adjust the selection option as per your needs
}

# Iterate over the folds
for train_index, val_index in kf.split(X_train):
    # Get the training and validation sets
    X_train_fold, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val = y_train.iloc[train_index], y_train.iloc[val_index]

    # Initialize the Elastic Net regression model
    en_model = ElasticNet()

    # Perform GridSearchCV to find the best parameters
    grid_search = GridSearchCV(en_model, en_param_grid, cv=n_splits)
    grid_search.fit(X_train_fold, y_train_fold)

    # Get the best model with the best parameters
    best_en_model = grid_search.best_estimator_

    # Fit the model on the training fold
    best_en_model.fit(X_train_fold, y_train_fold)

    # Evaluate the model on the validation set
    y_pred_val = best_en_model.predict(X_val)

    # Calculate the evaluation metrics (e.g., RMSE, MAE, R2 score, explained variance score) for each target variable
    for i, target_variable in enumerate(target_variables):
        en_rmse_val = np.sqrt(mean_squared_error(y_val[target_variable], y_pred_val[:, i]))
        en_mae_val = mean_absolute_error(y_val[target_variable], y_pred_val[:, i])
        en_r2_val = r2_score(y_val[target_variable], y_pred_val[:, i])
        en_explained_variance_val = explained_variance_score(y_val[target_variable], y_pred_val[:, i])

        # Append the errors to the respective lists
        en_rmse_list.append(en_rmse_val)
        en_mae_list.append(en_mae_val)
        en_r2_list.append(en_r2_val)
        en_explained_variance_list.append(en_explained_variance_val)

        # Append the errors to the DataFrame
        error_df = error_df.append({
            'Target Variable': target_variable,
            'RMSE': en_rmse_val,
            'MAE': en_mae_val,
            'R2 Score': en_r2_val,
            'Explained Variance': en_explained_variance_val
        }, ignore_index=True)

        print("Elastic Net RMSE for", target_variable, ":", en_rmse_val)
        print("Elastic Net MAE for", target_variable, ":", en_mae_val)
        print("Elastic Net R2 score for", target_variable, ":", en_r2_val)
        print("Elastic Net Explained variance score for", target_variable, ":", en_explained_variance_val)







<ipython-input-84-f6f26950785f>:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-84-f6f26950785f>:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-84-f6f26950785f>:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-84-f6f26950785f>:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-84-f6f26950785f>:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({


Elastic Net RMSE for p20_1200LR : 0.07901461794533617
Elastic Net MAE for p20_1200LR : 0.05539921064536143
Elastic Net R2 score for p20_1200LR : 0.5225316897925134
Elastic Net Explained variance score for p20_1200LR : 0.5271026841994981
Elastic Net RMSE for p20_1200LIB : 0.07901461796029212
Elastic Net MAE for p20_1200LIB : 0.05539921062898765
Elastic Net R2 score for p20_1200LIB : 0.5225316892858374
Elastic Net Explained variance score for p20_1200LIB : 0.5271026837388655
Elastic Net RMSE for P20_1200DT : 0.08411687987301153
Elastic Net MAE for P20_1200DT : 0.060251465211233875
Elastic Net R2 score for P20_1200DT : 0.5502564929497784
Elastic Net Explained variance score for P20_1200DT : 0.5515801109859197
Elastic Net RMSE for P20_1200ridge : 0.09278770670575562
Elastic Net MAE for P20_1200ridge : 0.0673101222121922
Elastic Net R2 score for P20_1200ridge : 0.5714329238313329
Elastic Net Explained variance score for P20_1200ridge : 0.5714362201233909
Elastic Net RMSE for Predicted_p20_1

<ipython-input-84-f6f26950785f>:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-84-f6f26950785f>:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-84-f6f26950785f>:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-84-f6f26950785f>:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-84-f6f26950785f>:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({


Elastic Net RMSE for p20_1200LR : 0.06816698720189242
Elastic Net MAE for p20_1200LR : 0.05376356988606861
Elastic Net R2 score for p20_1200LR : 0.6308962950088122
Elastic Net Explained variance score for p20_1200LR : 0.6813363271113875
Elastic Net RMSE for p20_1200LIB : 0.06816698725828839
Elastic Net MAE for p20_1200LIB : 0.05376356991417083
Elastic Net R2 score for p20_1200LIB : 0.6308962941314111
Elastic Net Explained variance score for p20_1200LIB : 0.6813363262885804
Elastic Net RMSE for P20_1200DT : 0.07166820025442046
Elastic Net MAE for P20_1200DT : 0.056977509118489214
Elastic Net R2 score for P20_1200DT : 0.6635017658541291
Elastic Net Explained variance score for P20_1200DT : 0.706617363130609
Elastic Net RMSE for P20_1200ridge : 0.07734284434755757
Elastic Net MAE for P20_1200ridge : 0.061328008146150984
Elastic Net R2 score for P20_1200ridge : 0.6929137645254915
Elastic Net Explained variance score for P20_1200ridge : 0.7284999391360829
Elastic Net RMSE for Predicted_p20_

<ipython-input-84-f6f26950785f>:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-84-f6f26950785f>:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-84-f6f26950785f>:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-84-f6f26950785f>:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-84-f6f26950785f>:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({


Elastic Net RMSE for p20_1200LR : 0.09365182132831228
Elastic Net MAE for p20_1200LR : 0.06455676662188231
Elastic Net R2 score for p20_1200LR : 0.6843156523184633
Elastic Net Explained variance score for p20_1200LR : 0.6875794256181122
Elastic Net RMSE for p20_1200LIB : 0.09365182124966445
Elastic Net MAE for p20_1200LIB : 0.06455676658683131
Elastic Net R2 score for p20_1200LIB : 0.6843156522208398
Elastic Net Explained variance score for p20_1200LIB : 0.6875794255646817
Elastic Net RMSE for P20_1200DT : 0.09357743238222943
Elastic Net MAE for P20_1200DT : 0.06722313187369819
Elastic Net R2 score for P20_1200DT : 0.7163881836577046
Elastic Net Explained variance score for P20_1200DT : 0.7182866735175557
Elastic Net RMSE for P20_1200ridge : 0.09754417761026044
Elastic Net MAE for P20_1200ridge : 0.07315255194040562
Elastic Net R2 score for P20_1200ridge : 0.740314148683459
Elastic Net Explained variance score for P20_1200ridge : 0.741199516202391
Elastic Net RMSE for Predicted_p20_120

<ipython-input-84-f6f26950785f>:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-84-f6f26950785f>:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-84-f6f26950785f>:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-84-f6f26950785f>:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-84-f6f26950785f>:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({


Elastic Net RMSE for p20_1200LR : 0.10835261345595602
Elastic Net MAE for p20_1200LR : 0.05836610640126652
Elastic Net R2 score for p20_1200LR : 0.6936560428888368
Elastic Net Explained variance score for p20_1200LR : 0.70611470686166
Elastic Net RMSE for p20_1200LIB : 0.10835261346286457
Elastic Net MAE for p20_1200LIB : 0.05836610633497035
Elastic Net R2 score for p20_1200LIB : 0.6936560424690257
Elastic Net Explained variance score for p20_1200LIB : 0.706114706506185
Elastic Net RMSE for P20_1200DT : 0.09562198196588755
Elastic Net MAE for P20_1200DT : 0.0566461914137064
Elastic Net R2 score for P20_1200DT : 0.765769286508336
Elastic Net Explained variance score for P20_1200DT : 0.7746722723394569
Elastic Net RMSE for P20_1200ridge : 0.0809578358215963
Elastic Net MAE for P20_1200ridge : 0.054524932766316925
Elastic Net R2 score for P20_1200ridge : 0.8404314370055166
Elastic Net Explained variance score for P20_1200ridge : 0.8449589934319549
Elastic Net RMSE for Predicted_p20_1200kn

<ipython-input-84-f6f26950785f>:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-84-f6f26950785f>:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-84-f6f26950785f>:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-84-f6f26950785f>:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-84-f6f26950785f>:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({


In [85]:
# Calculate the mean errors for each target variable
en_rmse_mean = np.mean(en_rmse_list)
en_mae_mean = np.mean(en_mae_list)
en_r2_mean = np.mean(en_r2_list)
en_explained_variance_mean = np.mean(en_explained_variance_list)

print("\nMean Elastic Net RMSE:", en_rmse_mean)
print("Mean Elastic Net MAE:", en_mae_mean)
print("Mean Elastic Net R2 score:", en_r2_mean)
print("Mean Elastic Net Explained variance score:", en_explained_variance_mean)

# Print the error DataFrame in a tabular format
print("\nError DataFrame:")
print(error_df)




Mean Elastic Net RMSE: 0.08635050253972762
Mean Elastic Net MAE: 0.06153753820345985
Mean Elastic Net R2 score: 0.6972098823124985
Mean Elastic Net Explained variance score: 0.7080989644075997

Error DataFrame:
          Target Variable      RMSE       MAE  R2 Score  Explained Variance
0              p20_1200LR  0.079015  0.055399  0.522532            0.527103
1             p20_1200LIB  0.079015  0.055399  0.522532            0.527103
2              P20_1200DT  0.084117  0.060251  0.550256            0.551580
3           P20_1200ridge  0.092788  0.067310  0.571433            0.571436
4   Predicted_p20_1200knn  0.092198  0.063946  0.546944            0.549069
5                    ANN2  0.094021  0.068377  0.573684            0.573688
6                      en  0.092214  0.066824  0.570321            0.570336
7              p20_1200LR  0.068167  0.053764  0.630896            0.681336
8             p20_1200LIB  0.068167  0.053764  0.630896            0.681336
9              P20_1200DT  0

In [86]:
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score
from sklearn.tree import DecisionTreeRegressor
import numpy as np
import pandas as pd

# Define lists to store the errors for each target variable
dt_rmse_list = []
dt_mae_list = []
dt_r2_list = []
dt_explained_variance_list = []

# Create a DataFrame to store the errors
error_df = pd.DataFrame(columns=['Target Variable', 'RMSE', 'MAE', 'R2 Score', 'Explained Variance'])

# Define the parameter grid for GridSearchCV
dt_param_grid = {
    'criterion': ['mse', 'friedman_mse', 'mae'],  # Adjust the criterion options as per your needs
    'max_depth': [None, 5, 10],  # Adjust the maximum depth options as per your needs
    'min_samples_split': [2, 5, 10],  # Adjust the minimum samples split options as per your needs
    'min_samples_leaf': [1, 2, 4],  # Adjust the minimum samples leaf options as per your needs
    'random_state': [42]  # Adjust the random state as per your needs
}

# Iterate over the folds
for train_index, val_index in kf.split(X_train):
    # Get the training and validation sets
    X_train_fold, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val = y_train.iloc[train_index], y_train.iloc[val_index]

    # Initialize the Decision Tree regression model
    dt_model = DecisionTreeRegressor()

    # Perform GridSearchCV to find the best parameters
    grid_search = GridSearchCV(dt_model, dt_param_grid, cv=n_splits)
    grid_search.fit(X_train_fold, y_train_fold)

    # Get the best model with the best parameters
    best_dt_model = grid_search.best_estimator_

    # Fit the model on the training fold
    best_dt_model.fit(X_train_fold, y_train_fold)

    # Evaluate the model on the validation set
    y_pred_val = best_dt_model.predict(X_val)

    # Calculate the evaluation metrics (e.g., RMSE, MAE, R2 score, explained variance score) for each target variable
    for i, target_variable in enumerate(target_variables):
        dt_rmse_val = np.sqrt(mean_squared_error(y_val[target_variable], y_pred_val[:, i]))
        dt_mae_val = mean_absolute_error(y_val[target_variable], y_pred_val[:, i])
        dt_r2_val = r2_score(y_val[target_variable], y_pred_val[:, i])
        dt_explained_variance_val = explained_variance_score(y_val[target_variable], y_pred_val[:, i])

        # Append the errors to the respective lists
        dt_rmse_list.append(dt_rmse_val)
        dt_mae_list.append(dt_mae_val)
        dt_r2_list.append(dt_r2_val)
        dt_explained_variance_list.append(dt_explained_variance_val)

        # Append the errors to the DataFrame
        error_df = error_df.append({
            'Target Variable': target_variable,
            'RMSE': dt_rmse_val,
            'MAE': dt_mae_val,
            'R2 Score': dt_r2_val,
            'Explained Variance': dt_explained_variance_val
        }, ignore_index=True)

        print("Decision Tree RMSE for", target_variable, ":", dt_rmse_val)
        print("Decision Tree MAE for", target_variable, ":", dt_mae_val)
        print("Decision Tree R2 score for", target_variable, ":", dt_r2_val)
        print("Decision Tree Explained variance score for", target_variable, ":", dt_explained_variance_val)



/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
270 fits failed out of a total of 405.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
135 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py", line 1247, in fit
    super().fit(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py", line 177, in fit
    self._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 600, in _val

Decision Tree RMSE for p20_1200LR : 0.051143837670669004
Decision Tree MAE for p20_1200LR : 0.03768175652346305
Decision Tree R2 score for p20_1200LR : 0.8492623699755035
Decision Tree Explained variance score for p20_1200LR : 0.8493745829060029
Decision Tree RMSE for p20_1200LIB : 0.05114383768415052
Decision Tree MAE for p20_1200LIB : 0.03768175648317966
Decision Tree R2 score for p20_1200LIB : 0.8492623696656401
Decision Tree Explained variance score for p20_1200LIB : 0.8493745825788164
Decision Tree RMSE for P20_1200DT : 0.05959628757925971
Decision Tree MAE for P20_1200DT : 0.04398823315684043
Decision Tree R2 score for P20_1200DT : 0.8218790780303071
Decision Tree Explained variance score for P20_1200DT : 0.8220178206307429
Decision Tree RMSE for P20_1200ridge : 0.07203818511387823
Decision Tree MAE for P20_1200ridge : 0.05266045131122997
Decision Tree R2 score for P20_1200ridge : 0.7864402758394866
Decision Tree Explained variance score for P20_1200ridge : 0.7865941569107856
Dec

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
270 fits failed out of a total of 405.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
135 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py", line 1247, in fit
    super().fit(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py", line 177, in fit
    self._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 600, in _val

Decision Tree RMSE for p20_1200LR : 0.07741978409843193
Decision Tree MAE for p20_1200LR : 0.04870638143622553
Decision Tree R2 score for p20_1200LR : 0.7069095125970319
Decision Tree Explained variance score for p20_1200LR : 0.7139793490344606
Decision Tree RMSE for p20_1200LIB : 0.07741978408747557
Decision Tree MAE for p20_1200LIB : 0.048706381344843
Decision Tree R2 score for p20_1200LIB : 0.7069095122876476
Decision Tree Explained variance score for p20_1200LIB : 0.713979348833918
Decision Tree RMSE for P20_1200DT : 0.08127236709918749
Decision Tree MAE for P20_1200DT : 0.052371132374745354
Decision Tree R2 score for P20_1200DT : 0.7258096318389009
Decision Tree Explained variance score for P20_1200DT : 0.73501577046587
Decision Tree RMSE for P20_1200ridge : 0.0890696110423616
Decision Tree MAE for P20_1200ridge : 0.058061763101475615
Decision Tree R2 score for P20_1200ridge : 0.7360149120292793
Decision Tree Explained variance score for P20_1200ridge : 0.7477645732019521
Decision

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
270 fits failed out of a total of 405.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
135 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py", line 1247, in fit
    super().fit(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py", line 177, in fit
    self._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 600, in _val

Decision Tree RMSE for p20_1200LR : 0.05305487764082854
Decision Tree MAE for p20_1200LR : 0.03551600677483285
Decision Tree R2 score for p20_1200LR : 0.8867743194440193
Decision Tree Explained variance score for p20_1200LR : 0.886817408093923
Decision Tree RMSE for p20_1200LIB : 0.05305487771349087
Decision Tree MAE for p20_1200LIB : 0.035516006800713325
Decision Tree R2 score for p20_1200LIB : 0.8867743190060046
Decision Tree Explained variance score for p20_1200LIB : 0.8868174076585906
Decision Tree RMSE for P20_1200DT : 0.05472065459550995
Decision Tree MAE for P20_1200DT : 0.037486902175443225
Decision Tree R2 score for P20_1200DT : 0.8876382119096954
Decision Tree Explained variance score for P20_1200DT : 0.8876460698661228
Decision Tree RMSE for P20_1200ridge : 0.0601126302201743
Decision Tree MAE for P20_1200ridge : 0.0407012052414805
Decision Tree R2 score for P20_1200ridge : 0.8800495551340759
Decision Tree Explained variance score for P20_1200ridge : 0.8802711795001937
Decis

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
270 fits failed out of a total of 405.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
135 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py", line 1247, in fit
    super().fit(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py", line 177, in fit
    self._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 600, in _val

Decision Tree RMSE for p20_1200LR : 0.08578044800035202
Decision Tree MAE for p20_1200LR : 0.041879729082947195
Decision Tree R2 score for p20_1200LR : 0.7804352307661029
Decision Tree Explained variance score for p20_1200LR : 0.8049447773280622
Decision Tree RMSE for p20_1200LIB : 0.08578044795688546
Decision Tree MAE for p20_1200LIB : 0.04187972909076806
Decision Tree R2 score for p20_1200LIB : 0.7804352305789624
Decision Tree Explained variance score for p20_1200LIB : 0.8049447773474346
Decision Tree RMSE for P20_1200DT : 0.08083588206752292
Decision Tree MAE for P20_1200DT : 0.04137218679326325
Decision Tree R2 score for P20_1200DT : 0.81279728596033
Decision Tree Explained variance score for P20_1200DT : 0.8349876477081166
Decision Tree RMSE for P20_1200ridge : 0.07685125084498753
Decision Tree MAE for P20_1200ridge : 0.04124173596280426
Decision Tree R2 score for P20_1200ridge : 0.844200867988284
Decision Tree Explained variance score for P20_1200ridge : 0.8633762380826723
Decisi

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
270 fits failed out of a total of 405.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
135 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py", line 1247, in fit
    super().fit(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py", line 177, in fit
    self._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 600, in _val

In [87]:
# Calculate the mean errors for each target variable
dt_rmse_mean = np.mean(dt_rmse_list)
dt_mae_mean = np.mean(dt_mae_list)
dt_r2_mean = np.mean(dt_r2_list)
dt_explained_variance_mean = np.mean(dt_explained_variance_list)

print("\nMean Decision Tree RMSE:", dt_rmse_mean)
print("Mean Decision Tree MAE:", dt_mae_mean)
print("Mean Decision Tree R2 score:", dt_r2_mean)
print("Mean Decision Tree Explained variance score:", dt_explained_variance_mean)

# Print the error DataFrame in a tabular format
print("\nError DataFrame:")
print(error_df)



Mean Decision Tree RMSE: 0.06586371739700628
Mean Decision Tree MAE: 0.0424835059776692
Mean Decision Tree R2 score: 0.8314336447925145
Mean Decision Tree Explained variance score: 0.8376400191641695

Error DataFrame:
          Target Variable      RMSE       MAE  R2 Score  Explained Variance
0              p20_1200LR  0.051144  0.037682  0.849262            0.849375
1             p20_1200LIB  0.051144  0.037682  0.849262            0.849375
2              P20_1200DT  0.059596  0.043988  0.821879            0.822018
3           P20_1200ridge  0.072038  0.052660  0.786440            0.786594
4   Predicted_p20_1200knn  0.064313  0.046575  0.818902            0.819159
5                    ANN2  0.073997  0.053931  0.780533            0.780683
6                      en  0.071189  0.052075  0.788788            0.788941
7              p20_1200LR  0.077420  0.048706  0.706910            0.713979
8             p20_1200LIB  0.077420  0.048706  0.706910            0.713979
9              P20_12

In [88]:
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score
from sklearn.linear_model import RANSACRegressor
import numpy as np
import pandas as pd


# Define lists to store the errors for each target variable
ransac_rmse_list = []
ransac_mae_list = []
ransac_r2_list = []
ransac_explained_variance_list = []

# Create a DataFrame to store the errors
error_df = pd.DataFrame(columns=['Target Variable', 'RMSE', 'MAE', 'R2 Score', 'Explained Variance'])

# Define the parameter grid for GridSearchCV
param_grid = {
    'base_estimator': [None],  # Adjust the base estimator as per your needs
    'min_samples': [0.1, 0.5, 1.0],  # Adjust the minimum number of samples as per your needs
    'residual_threshold': [0.1, 1.0, 5.0],  # Adjust the residual threshold as per your needs
    'max_trials': [100, 500, 1000],  # Adjust the maximum number of trials as per your needs
    'max_skips': [np.inf],  # Adjust the maximum number of skips as per your needs
    'stop_probability': [0.99],  # Adjust the stop probability as per your needs
    'stop_score': [np.inf],  # Adjust the stop score as per your needs
    'loss': ['absolute_error', 'squared_error'],  # Adjust the loss function as per your needs
    'random_state': [42]  # Adjust the random state as per your needs
}

# Iterate over the folds
for train_index, val_index in kf.split(X_train):
    # Get the training and validation sets
    X_train_fold, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val = y_train.iloc[train_index], y_train.iloc[val_index]

    # Initialize the RANSAC regression model
    ransac_model = RANSACRegressor()

    # Perform GridSearchCV to find the best parameters
    grid_search = GridSearchCV(ransac_model, param_grid, cv=n_splits)
    grid_search.fit(X_train_fold, y_train_fold)

    # Get the best model with the best parameters
    best_ransac_model = grid_search.best_estimator_

    # Evaluate the model on the validation set
    y_pred_val = best_ransac_model.predict(X_val)

    # Calculate the evaluation metrics (e.g., RMSE, MAE, R2 score, explained variance score) for each target variable
    for i, target_variable in enumerate(target_variables):
        ransac_rmse_val = np.sqrt(mean_squared_error(y_val[target_variable], y_pred_val[:, i]))
        ransac_mae_val = mean_absolute_error(y_val[target_variable], y_pred_val[:, i])
        ransac_r2_val = r2_score(y_val[target_variable], y_pred_val[:, i])
        ransac_explained_variance_val = explained_variance_score(y_val[target_variable], y_pred_val[:, i])

        # Append the errors to the respective lists
        ransac_rmse_list.append(ransac_rmse_val)
        ransac_mae_list.append(ransac_mae_val)
        ransac_r2_list.append(ransac_r2_val)
        ransac_explained_variance_list.append(ransac_explained_variance_val)

        # Append the errors to the DataFrame
        error_df = error_df.append({
            'Target Variable': target_variable,
            'RMSE': ransac_rmse_val,
            'MAE': ransac_mae_val,
            'R2 Score': ransac_r2_val,
            'Explained Variance': ransac_explained_variance_val
        }, ignore_index=True)

        print("RANSAC RMSE for", target_variable, ":", ransac_rmse_val)
        print("RANSAC MAE for", target_variable, ":", ransac_mae_val)
        print("RANSAC R2 score for", target_variable, ":", ransac_r2_val)
        print("RANSAC Explained variance score for", target_variable, ":", ransac_explained_variance_val)






/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
/usr/local/lib/pytho

RANSAC RMSE for p20_1200LR : 0.0358150643275464
RANSAC MAE for p20_1200LR : 0.025228874111776642
RANSAC R2 score for p20_1200LR : 0.9387803873265671
RANSAC Explained variance score for p20_1200LR : 0.9399159274582486
RANSAC RMSE for p20_1200LIB : 0.03581506433809886
RANSAC MAE for p20_1200LIB : 0.025228874097231548
RANSAC R2 score for p20_1200LIB : 0.938780387192538
RANSAC Explained variance score for p20_1200LIB : 0.939915927351238
RANSAC RMSE for P20_1200DT : 0.03553971957037045
RANSAC MAE for P20_1200DT : 0.02597600959291493
RANSAC R2 score for P20_1200DT : 0.9459188863803748
RANSAC Explained variance score for P20_1200DT : 0.9484019431515207
RANSAC RMSE for P20_1200ridge : 0.038743383064777134
RANSAC MAE for P20_1200ridge : 0.028959719159914254
RANSAC R2 score for P20_1200ridge : 0.9455757758928378
RANSAC Explained variance score for P20_1200ridge : 0.9501986869289282
RANSAC RMSE for Predicted_p20_1200knn : 0.0422791242662891
RANSAC MAE for Predicted_p20_1200knn : 0.030802050834010

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
/usr/local/lib/pytho

RANSAC RMSE for p20_1200LR : 0.03851367662715452
RANSAC MAE for p20_1200LR : 0.024344305338841403
RANSAC R2 score for p20_1200LR : 0.9478225509797354
RANSAC Explained variance score for p20_1200LR : 0.9503241310797114
RANSAC RMSE for p20_1200LIB : 0.03851367669486741
RANSAC MAE for p20_1200LIB : 0.024344305348287583
RANSAC R2 score for p20_1200LIB : 0.947822550760183
RANSAC Explained variance score for p20_1200LIB : 0.9503241308811239
RANSAC RMSE for P20_1200DT : 0.0397054740647974
RANSAC MAE for P20_1200DT : 0.024857465285982004
RANSAC R2 score for P20_1200DT : 0.9510926330003029
RANSAC Explained variance score for P20_1200DT : 0.9529976068584081
RANSAC RMSE for P20_1200ridge : 0.04443438429057112
RANSAC MAE for P20_1200ridge : 0.02935360677387721
RANSAC R2 score for P20_1200ridge : 0.9490790431684466
RANSAC Explained variance score for P20_1200ridge : 0.9505892262980328
RANSAC RMSE for Predicted_p20_1200knn : 0.05133075645639195
RANSAC MAE for Predicted_p20_1200knn : 0.03088313048633

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
/usr/local/lib/pytho

...

<ipython-input-88-c9672a9838a4>:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-88-c9672a9838a4>:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-88-c9672a9838a4>:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-88-c9672a9838a4>:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-88-c9672a9838a4>:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({


RANSAC RMSE for p20_1200LR : 0.03949741936489207
RANSAC MAE for p20_1200LR : 0.027562881167448844
RANSAC R2 score for p20_1200LR : 0.9423132495470471
RANSAC Explained variance score for p20_1200LR : 0.9431207906102957
RANSAC RMSE for p20_1200LIB : 0.039497419381558756
RANSAC MAE for p20_1200LIB : 0.027562881089628576
RANSAC R2 score for p20_1200LIB : 0.9423132494084114
RANSAC Explained variance score for p20_1200LIB : 0.9431207904817275
RANSAC RMSE for P20_1200DT : 0.0356263045263431
RANSAC MAE for P20_1200DT : 0.025203003151631386
RANSAC R2 score for P20_1200DT : 0.9562517357649587
RANSAC Explained variance score for P20_1200DT : 0.9567020557364516
RANSAC RMSE for P20_1200ridge : 0.0347919511690218
RANSAC MAE for P20_1200ridge : 0.026681484815798566
RANSAC R2 score for P20_1200ridge : 0.9630941256964167
RANSAC Explained variance score for P20_1200ridge : 0.9631964671829596
RANSAC RMSE for Predicted_p20_1200knn : 0.0379991144338003
RANSAC MAE for Predicted_p20_1200knn : 0.0277363749677

<ipython-input-88-c9672a9838a4>:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-88-c9672a9838a4>:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-88-c9672a9838a4>:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-88-c9672a9838a4>:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-88-c9672a9838a4>:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({


RANSAC RMSE for p20_1200LR : 0.03241923831911146
RANSAC MAE for p20_1200LR : 0.024302908991442784
RANSAC R2 score for p20_1200LR : 0.9261702492637216
RANSAC Explained variance score for p20_1200LR : 0.9282667695248706
RANSAC RMSE for p20_1200LIB : 0.03241923836283275
RANSAC MAE for p20_1200LIB : 0.02430290905760862
RANSAC R2 score for p20_1200LIB : 0.926170248952687
RANSAC Explained variance score for p20_1200LIB : 0.9282667692769544
RANSAC RMSE for P20_1200DT : 0.0324451480418877
RANSAC MAE for P20_1200DT : 0.024796524386330322
RANSAC R2 score for P20_1200DT : 0.9374421551904455
RANSAC Explained variance score for P20_1200DT : 0.9402288054010051
RANSAC RMSE for P20_1200ridge : 0.03515361842769088
RANSAC MAE for P20_1200ridge : 0.02676457131440767
RANSAC R2 score for P20_1200ridge : 0.9411272962969494
RANSAC Explained variance score for P20_1200ridge : 0.9450905744616035
RANSAC RMSE for Predicted_p20_1200knn : 0.04002967648346514
RANSAC MAE for Predicted_p20_1200knn : 0.028520290149244

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ransac.py:343: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.1 and will be removed in 1.3.
  warnings.warn(
/usr/local/lib/pytho

In [89]:
# Calculate the mean errors for each target variable
mean_rmse = np.mean(ransac_rmse_list)
mean_mae = np.mean(ransac_mae_list)
mean_r2 = np.mean(ransac_r2_list)
mean_explained_variance = np.mean(ransac_explained_variance_list)

print("\nMean RMSE:", mean_rmse)
print("Mean MAE:", mean_mae)
print("Mean R2 score:", mean_r2)
print("Mean Explained variance score:", mean_explained_variance)

# Print the error DataFrame in a tabular format
print(error_df)


Mean RMSE: 0.03767790254547021
Mean MAE: 0.026439971683011463
Mean R2 score: 0.9465378010769655
Mean Explained variance score: 0.9486968285552072
          Target Variable      RMSE       MAE  R2 Score  Explained Variance
0              p20_1200LR  0.035815  0.025229  0.938780            0.939916
1             p20_1200LIB  0.035815  0.025229  0.938780            0.939916
2              P20_1200DT  0.035540  0.025976  0.945919            0.948402
3           P20_1200ridge  0.038743  0.028960  0.945576            0.950199
4   Predicted_p20_1200knn  0.042279  0.030802  0.933359            0.936030
5                    ANN2  0.039362  0.029584  0.945066            0.949976
6                      en  0.038479  0.028660  0.945747            0.950220
7              p20_1200LR  0.038514  0.024344  0.947823            0.950324
8             p20_1200LIB  0.038514  0.024344  0.947823            0.950324
9              P20_1200DT  0.039705  0.024857  0.951093            0.952998
10          P20_1

In [90]:
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score
import numpy as np
import pandas as pd


# Define lists to store the errors for each target variable
gpr_rmse_list = []
gpr_mae_list = []
gpr_r2_list = []
gpr_explained_variance_list = []

# Create a DataFrame to store the errors
error_df = pd.DataFrame(columns=['Target Variable', 'RMSE', 'MAE', 'R2 Score', 'Explained Variance'])

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'kernel': [RBF(length_scale=1.0, length_scale_bounds=(1e-1, 10.0)),
               RBF(length_scale=0.5, length_scale_bounds=(1e-1, 10.0)),
               RBF(length_scale=2.0, length_scale_bounds=(1e-1, 10.0))],
    'alpha': [1e-5, 1e-4, 1e-3],
    'optimizer': ['fmin_l_bfgs_b', 'fmin_ncg'],
    'n_restarts_optimizer': [0, 5, 10]
}

# Iterate over the folds
for train_index, val_index in kf.split(X_train):
    # Get the training and validation sets
    X_train_fold, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val = y_train.iloc[train_index], y_train.iloc[val_index]

    # Initialize the Gaussian Process Regression model
    gpr_model = GaussianProcessRegressor()

    # Perform GridSearchCV to find the best parameters
    grid_search = GridSearchCV(gpr_model, param_grid, cv=n_splits)
    grid_search.fit(X_train_fold, y_train_fold)

    # Get the best model with the best parameters
    best_gpr_model = grid_search.best_estimator_

    # Fit the best model on the training fold
    best_gpr_model.fit(X_train_fold, y_train_fold)

    # Evaluate the best model on the validation set
    y_pred_val = best_gpr_model.predict(X_val)

    # Calculate the evaluation metrics (e.g., RMSE, MAE, R2 score, explained variance score) for each target variable
    for i, target_variable in enumerate(target_variables):
        gpr_rmse_val = np.sqrt(mean_squared_error(y_val[target_variable], y_pred_val[:, i]))
        gpr_mae_val = mean_absolute_error(y_val[target_variable], y_pred_val[:, i])
        gpr_r2_val = r2_score(y_val[target_variable], y_pred_val[:, i])
        gpr_explained_variance_val = explained_variance_score(y_val[target_variable], y_pred_val[:, i])

        # Append the errors to the respective lists
        gpr_rmse_list.append(gpr_rmse_val)
        gpr_mae_list.append(gpr_mae_val)
        gpr_r2_list.append(gpr_r2_val)
        gpr_explained_variance_list.append(gpr_explained_variance_val)

        # Append the errors to the DataFrame
        error_df = error_df.append({
            'Target Variable': target_variable,
            'RMSE': gpr_rmse_val,
            'MAE': gpr_mae_val,
            'R2 Score': gpr_r2_val,
            'Explained Variance': gpr_explained_variance_val
        }, ignore_index=True)

        print("GPR RMSE for", target_variable, ":", gpr_rmse_val)
        print("GPR MAE for", target_variable, ":", gpr_mae_val)
        print("GPR R2 score for", target_variable, ":", gpr_r2_val)
        print("GPR Explained variance score for", target_variable, ":", gpr_explained_variance_val)



/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessin

GPR RMSE for p20_1200LR : 0.053562842529028654
GPR MAE for p20_1200LR : 0.032274250206233604
GPR R2 score for p20_1200LR : 0.8975756672637398
GPR Explained variance score for p20_1200LR : 0.909156336363944
GPR RMSE for p20_1200LIB : 0.05356284267799386
GPR MAE for p20_1200LIB : 0.03227425032012253
GPR R2 score for p20_1200LIB : 0.897575666630226
GPR Explained variance score for p20_1200LIB : 0.9091563358019632
GPR RMSE for P20_1200DT : 0.05258699993823921
GPR MAE for P20_1200DT : 0.031407867683982925
GPR R2 score for P20_1200DT : 0.9067335785040893
GPR Explained variance score for P20_1200DT : 0.9165086968819787
GPR RMSE for P20_1200ridge : 0.05456812624622607
GPR MAE for P20_1200ridge : 0.03209785616412393
GPR R2 score for P20_1200ridge : 0.9097515746836498
GPR Explained variance score for P20_1200ridge : 0.9175384342071122
GPR RMSE for Predicted_p20_1200knn : 0.06197013602607398
GPR MAE for Predicted_p20_1200knn : 0.03512098821853638
GPR R2 score for Predicted_p20_1200knn : 0.8858110

/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessin

GPR RMSE for p20_1200LR : 0.02304484135986056
GPR MAE for p20_1200LR : 0.018506764102517476
GPR R2 score for p20_1200LR : 0.9685700204543808
GPR Explained variance score for p20_1200LR : 0.9686074087397157
GPR RMSE for p20_1200LIB : 0.023044841356060757
GPR MAE for p20_1200LIB : 0.0185067641119614
GPR R2 score for p20_1200LIB : 0.9685700204194139
GPR Explained variance score for p20_1200LIB : 0.9686074087103014
GPR RMSE for P20_1200DT : 0.022177638861030705
GPR MAE for P20_1200DT : 0.018270367251168054
GPR R2 score for P20_1200DT : 0.9743197896127248
GPR Explained variance score for P20_1200DT : 0.9743580910303472
GPR RMSE for P20_1200ridge : 0.023259330682952422
GPR MAE for P20_1200ridge : 0.01856797078232969
GPR R2 score for P20_1200ridge : 0.976603009681819
GPR Explained variance score for P20_1200ridge : 0.976642789822634
GPR RMSE for Predicted_p20_1200knn : 0.027803526954213446
GPR MAE for Predicted_p20_1200knn : 0.02173726951051332
GPR R2 score for Predicted_p20_1200knn : 0.96637

/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessin

...

/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessin

GPR RMSE for p20_1200LR : 0.02863504550860867
GPR MAE for p20_1200LR : 0.02186005630240359
GPR R2 score for p20_1200LR : 0.9137757674415996
GPR Explained variance score for p20_1200LR : 0.9305965354070153
GPR RMSE for p20_1200LIB : 0.028635045592026856
GPR MAE for p20_1200LIB : 0.021860056358543517
GPR R2 score for p20_1200LIB : 0.9137757667949462
GPR Explained variance score for p20_1200LIB : 0.9305965350631782
GPR RMSE for P20_1200DT : 0.02860848405887383
GPR MAE for P20_1200DT : 0.02189311897776903
GPR R2 score for P20_1200DT : 0.933457784445438
GPR Explained variance score for P20_1200DT : 0.9461372431300761
GPR RMSE for P20_1200ridge : 0.030028224026853616
GPR MAE for P20_1200ridge : 0.022927355171225787
GPR R2 score for P20_1200ridge : 0.9459426853020197
GPR Explained variance score for P20_1200ridge : 0.9549876458703586
GPR RMSE for Predicted_p20_1200knn : 0.03467378905876974
GPR MAE for Predicted_p20_1200knn : 0.026266784729496267
GPR R2 score for Predicted_p20_1200knn : 0.9109

<ipython-input-90-002357c47d69>:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-90-002357c47d69>:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-90-002357c47d69>:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-90-002357c47d69>:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-90-002357c47d69>:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({


GPR RMSE for p20_1200LR : 0.037683815206780547
GPR MAE for p20_1200LR : 0.025472222520018784
GPR R2 score for p20_1200LR : 0.9081026143498118
GPR Explained variance score for p20_1200LR : 0.9164538855271375
GPR RMSE for p20_1200LIB : 0.037683815204148374
GPR MAE for p20_1200LIB : 0.025472222401144985
GPR R2 score for p20_1200LIB : 0.9081026142853565
GPR Explained variance score for p20_1200LIB : 0.9164538856375675
GPR RMSE for P20_1200DT : 0.037074367439415674
GPR MAE for P20_1200DT : 0.025369770828787698
GPR R2 score for P20_1200DT : 0.9235779990932279
GPR Explained variance score for P20_1200DT : 0.9313971242516262
GPR RMSE for P20_1200ridge : 0.03804492999514141
GPR MAE for P20_1200ridge : 0.026010777681460664
GPR R2 score for P20_1200ridge : 0.9349010487185904
GPR Explained variance score for P20_1200ridge : 0.9423096383953249
GPR RMSE for Predicted_p20_1200knn : 0.04704587117676428
GPR MAE for Predicted_p20_1200knn : 0.03030514679261496
GPR R2 score for Predicted_p20_1200knn : 0.8

<ipython-input-90-002357c47d69>:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-90-002357c47d69>:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-90-002357c47d69>:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-90-002357c47d69>:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({
<ipython-input-90-002357c47d69>:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_df = error_df.append({


In [91]:
# Calculate the mean errors for each target variable
mean_rmse = np.mean(gpr_rmse_list)
mean_mae = np.mean(gpr_mae_list)
mean_r2 = np.mean(gpr_r2_list)
mean_explained_variance = np.mean(gpr_explained_variance_list)

print("\nMean RMSE:", mean_rmse)
print("Mean MAE:", mean_mae)
print("Mean R2 score:", mean_r2)
print("Mean Explained variance score:", mean_explained_variance)

# Print the error DataFrame in a tabular format
print(error_df)



Mean RMSE: 0.037611570486429186
Mean MAE: 0.025134689876742382
Mean R2 score: 0.9382953506313138
Mean Explained variance score: 0.9451919884809599
          Target Variable      RMSE       MAE  R2 Score  Explained Variance
0              p20_1200LR  0.053563  0.032274  0.897576            0.909156
1             p20_1200LIB  0.053563  0.032274  0.897576            0.909156
2              P20_1200DT  0.052587  0.031408  0.906734            0.916509
3           P20_1200ridge  0.054568  0.032098  0.909752            0.917538
4   Predicted_p20_1200knn  0.061970  0.035121  0.885811            0.894968
5                    ANN2  0.054557  0.032116  0.910973            0.918505
6                      en  0.054506  0.032076  0.909343            0.917261
7              p20_1200LR  0.023045  0.018507  0.968570            0.968607
8             p20_1200LIB  0.023045  0.018507  0.968570            0.968607
9              P20_1200DT  0.022178  0.018270  0.974320            0.974358
10          P20_

In [92]:
y_pred_test = best_model.predict(X_test)
# Create a DataFrame to store the evaluation metrics
evaluation_df = pd.DataFrame(columns=['Target Variable', 'RMSE', 'MAE', 'R2 Score', 'Explained Variance'])

# Calculate the evaluation metrics for all target variables
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test, multioutput='raw_values'))
mae_test = mean_absolute_error(y_test, y_pred_test, multioutput='raw_values')
r2_test = r2_score(y_test, y_pred_test, multioutput='raw_values')
explained_variance_test = explained_variance_score(y_test, y_pred_test, multioutput='raw_values')

# Print the evaluation metrics for all target variables
for i, target_variable in enumerate(target_variables):
    print("Linear Regression Test RMSE for", target_variable, ":", rmse_test[i])
    print("Linear Regression Test MAE for", target_variable, ":", mae_test[i])
    print("Linear Regression Test R2 Score for", target_variable, ":", r2_test[i])
    print("Linear Regression Test Explained Variance for", target_variable, ":", explained_variance_test[i])
    print()

# Calculate the mean of all errors
mean_rmse = np.mean(rmse_test)
mean_mae = np.mean(mae_test)
mean_r2 = np.mean(r2_test)
mean_explained_variance = np.mean(explained_variance_test)



Linear Regression Test RMSE for p20_1200LR : 0.04950040622817607
Linear Regression Test MAE for p20_1200LR : 0.032829709569466264
Linear Regression Test R2 Score for p20_1200LR : 0.9168748943708035
Linear Regression Test Explained Variance for p20_1200LR : 0.9183789468531148

Linear Regression Test RMSE for p20_1200LIB : 0.049500406154558925
Linear Regression Test MAE for p20_1200LIB : 0.032829709517785556
Linear Regression Test R2 Score for p20_1200LIB : 0.9168748944608832
Linear Regression Test Explained Variance for p20_1200LIB : 0.9183789469317988

Linear Regression Test RMSE for P20_1200DT : 0.04752541420090719
Linear Regression Test MAE for P20_1200DT : 0.03201853599382026
Linear Regression Test R2 Score for P20_1200DT : 0.929383266689212
Linear Regression Test Explained Variance for P20_1200DT : 0.930043278541823

Linear Regression Test RMSE for P20_1200ridge : 0.04823282948976483
Linear Regression Test MAE for P20_1200ridge : 0.03287328570087424
Linear Regression Test R2 Score 

In [93]:
# Print the mean errors
print("Mean RMSE:", mean_rmse)
print("Mean MAE:", mean_mae)
print("Mean R2 Score:", mean_r2)
print("Mean Explained Variance:", mean_explained_variance)


Mean RMSE: 0.05042513824009231
Mean MAE: 0.03382418023973654
Mean R2 Score: 0.9251676537051177
Mean Explained Variance: 0.9259671587988202


In [94]:
# Test the Lasso model
y_pred_lasso = best_lasso_model.predict(X_test)

# Create a DataFrame to store the test errors
test_error_df = pd.DataFrame(columns=['Target Variable', 'RMSE', 'MAE', 'R2 Score', 'Explained Variance'])

# Calculate the test errors for each target variable
lasso_rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_lasso, multioutput='raw_values'))
lasso_mae_test = mean_absolute_error(y_test, y_pred_lasso, multioutput='raw_values')
lasso_r2_test = r2_score(y_test, y_pred_lasso, multioutput='raw_values')
lasso_explained_variance_test = explained_variance_score(y_test, y_pred_lasso, multioutput='raw_values')

# Print the test errors for each target variable
for i, target_variable in enumerate(target_variables):
    print("Lasso Test RMSE for", target_variable, ":", lasso_rmse_test[i])
    print("Lasso Test MAE for", target_variable, ":", lasso_mae_test[i])
    print("Lasso Test R2 Score for", target_variable, ":", lasso_r2_test[i])
    print("Lasso Test Explained Variance for", target_variable, ":", lasso_explained_variance_test[i])
    print()

    # Append the test errors to the DataFrame
    test_error_df = test_error_df.append({
        'Target Variable': target_variable,
        'RMSE': lasso_rmse_test[i],
        'MAE': lasso_mae_test[i],
        'R2 Score': lasso_r2_test[i],
        'Explained Variance': lasso_explained_variance_test[i]
    }, ignore_index=True)


# Print the test error DataFrame in a tabular format
print(test_error_df)


Lasso Test RMSE for p20_1200LR : 0.16016866889505602
Lasso Test MAE for p20_1200LR : 0.11168329519669169
Lasso Test R2 Score for p20_1200LR : 0.12969833014956234
Lasso Test Explained Variance for p20_1200LR : 0.1510787229620898

Lasso Test RMSE for p20_1200LIB : 0.16016866879702973
Lasso Test MAE for p20_1200LIB : 0.11168329514445365
Lasso Test R2 Score for p20_1200LIB : 0.12969832956932859
Lasso Test Explained Variance for p20_1200LIB : 0.1510787223636707

Lasso Test RMSE for P20_1200DT : 0.1607494600989224
Lasso Test MAE for P20_1200DT : 0.11681804503425847
Lasso Test R2 Score for P20_1200DT : 0.19210513412829178
Lasso Test Explained Variance for P20_1200DT : 0.2140259849017122

Lasso Test RMSE for P20_1200ridge : 0.16363534732037724
Lasso Test MAE for P20_1200ridge : 0.12403031186333299
Lasso Test R2 Score for P20_1200ridge : 0.26807369799631386
Lasso Test Explained Variance for P20_1200ridge : 0.29035240178302035

Lasso Test RMSE for Predicted_p20_1200knn : 0.17429171321778045
Lass

<ipython-input-94-4ca0665102f6>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
<ipython-input-94-4ca0665102f6>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
<ipython-input-94-4ca0665102f6>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
<ipython-input-94-4ca0665102f6>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
<ipython-input-94-4ca0665102f6>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat 

In [95]:
# Calculate the average of all errors
mean_rmse = np.mean(lasso_rmse_test)
mean_mae = np.mean(lasso_mae_test)
mean_r2 = np.mean(lasso_r2_test)
mean_explained_variance = np.mean(lasso_explained_variance_test)

# Print the average errors
print("Mean RMSE:", mean_rmse)
print("Mean MAE:", mean_mae)
print("Mean R2 Score:", mean_r2)
print("Mean Explained Variance:", mean_explained_variance)


Mean RMSE: 0.16377657617643915
Mean MAE: 0.11922530568030519
Mean R2 Score: 0.21281321619309618
Mean Explained Variance: 0.235222761909943


In [96]:
# Test the Ridge model
y_pred_ridge = best_ridge_model.predict(X_test)

# Create a DataFrame to store the test errors
test_error_df = pd.DataFrame(columns=['Target Variable', 'RMSE', 'MAE', 'R2 Score', 'Explained Variance'])

# Calculate the test errors for each target variable
ridge_rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_ridge, multioutput='raw_values'))
ridge_mae_test = mean_absolute_error(y_test, y_pred_ridge, multioutput='raw_values')
ridge_r2_test = r2_score(y_test, y_pred_ridge, multioutput='raw_values')
ridge_explained_variance_test = explained_variance_score(y_test, y_pred_ridge, multioutput='raw_values')

# Print the test errors for each target variable
for i, target_variable in enumerate(target_variables):
    print("Ridge Test RMSE for", target_variable, ":", ridge_rmse_test[i])
    print("Ridge Test MAE for", target_variable, ":", ridge_mae_test[i])
    print("Ridge Test R2 Score for", target_variable, ":", ridge_r2_test[i])
    print("Ridge Test Explained Variance for", target_variable, ":", ridge_explained_variance_test[i])
    print()

    # Append the test errors to the DataFrame
    test_error_df = test_error_df.append({
        'Target Variable': target_variable,
        'RMSE': ridge_rmse_test[i],
        'MAE': ridge_mae_test[i],
        'R2 Score': ridge_r2_test[i],
        'Explained Variance': ridge_explained_variance_test[i]
    }, ignore_index=True)



# Print the test error DataFrame in a tabular format
print(test_error_df)


Ridge Test RMSE for p20_1200LR : 0.04942379211845234
Ridge Test MAE for p20_1200LR : 0.032832320870647365
Ridge Test R2 Score for p20_1200LR : 0.9171320085236514
Ridge Test Explained Variance for p20_1200LR : 0.9176839705408286

Ridge Test RMSE for p20_1200LIB : 0.04942379204443903
Ridge Test MAE for p20_1200LIB : 0.03283232080708448
Ridge Test R2 Score for p20_1200LIB : 0.917132008615163
Ridge Test Explained Variance for p20_1200LIB : 0.9176839706291257

Ridge Test RMSE for P20_1200DT : 0.047279373460972655
Ridge Test MAE for P20_1200DT : 0.031750949160729985
Ridge Test R2 Score for P20_1200DT : 0.9301125446625554
Ridge Test Explained Variance for P20_1200DT : 0.9304389386549112

Ridge Test RMSE for P20_1200ridge : 0.04723231685522931
Ridge Test MAE for P20_1200ridge : 0.0322327532113927
Ridge Test R2 Score for P20_1200ridge : 0.9390194120645816
Ridge Test Explained Variance for P20_1200ridge : 0.9391549689175792

Ridge Test RMSE for Predicted_p20_1200knn : 0.060641146647777096
Ridge 

<ipython-input-96-bb82f67395cb>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
<ipython-input-96-bb82f67395cb>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
<ipython-input-96-bb82f67395cb>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
<ipython-input-96-bb82f67395cb>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
<ipython-input-96-bb82f67395cb>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat 

In [97]:
# Calculate the average of all errors
mean_rmse = np.mean(ridge_rmse_test)
mean_mae = np.mean(ridge_mae_test)
mean_r2 = np.mean(ridge_r2_test)
mean_explained_variance = np.mean(ridge_explained_variance_test)

# Print the average errors
print("Mean RMSE:", mean_rmse)
print("Mean MAE:", mean_mae)
print("Mean R2 Score:", mean_r2)
print("Mean Explained Variance:", mean_explained_variance)

Mean RMSE: 0.04978281527594175
Mean MAE: 0.0334271935447177
Mean R2 Score: 0.926940544981848
Mean Explained Variance: 0.9273409236432858


In [98]:
# Test the KNN model
y_pred_knn = best_knn_model.predict(X_test)

# Create a DataFrame to store the test errors
test_error_df = pd.DataFrame(columns=['Target Variable', 'RMSE', 'MAE', 'R2 Score', 'Explained Variance'])

# Calculate the test errors for each target variable
knn_rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_knn, multioutput='raw_values'))
knn_mae_test = mean_absolute_error(y_test, y_pred_knn, multioutput='raw_values')
knn_r2_test = r2_score(y_test, y_pred_knn, multioutput='raw_values')
knn_explained_variance_test = explained_variance_score(y_test, y_pred_knn, multioutput='raw_values')

# Print the test errors for each target variable
for i, target_variable in enumerate(target_variables):
    print("KNN Test RMSE for", target_variable, ":", knn_rmse_test[i])
    print("KNN Test MAE for", target_variable, ":", knn_mae_test[i])
    print("KNN Test R2 Score for", target_variable, ":", knn_r2_test[i])
    print("KNN Test Explained Variance for", target_variable, ":", knn_explained_variance_test[i])
    print()

    # Append the test errors to the DataFrame
    test_error_df = test_error_df.append({
        'Target Variable': target_variable,
        'RMSE': knn_rmse_test[i],
        'MAE': knn_mae_test[i],
        'R2 Score': knn_r2_test[i],
        'Explained Variance': knn_explained_variance_test[i]
    }, ignore_index=True)



# Print the test error DataFrame in a tabular format
print(test_error_df)


KNN Test RMSE for p20_1200LR : 0.06367116144404385
KNN Test MAE for p20_1200LR : 0.03188401029698024
KNN Test R2 Score for p20_1200LR : 0.862469117989446
KNN Test Explained Variance for p20_1200LR : 0.8668752782962744

KNN Test RMSE for p20_1200LIB : 0.06367116132540296
KNN Test MAE for p20_1200LIB : 0.03188401023300532
KNN Test R2 Score for p20_1200LIB : 0.8624691182419436
KNN Test Explained Variance for p20_1200LIB : 0.8668752785603333

KNN Test RMSE for P20_1200DT : 0.06351461010952554
KNN Test MAE for P20_1200DT : 0.03314125215584451
KNN Test R2 Score for P20_1200DT : 0.8738744493702928
KNN Test Explained Variance for P20_1200DT : 0.8776524423515664

KNN Test RMSE for P20_1200ridge : 0.06491797083308047
KNN Test MAE for P20_1200ridge : 0.03526492105366965
KNN Test R2 Score for P20_1200ridge : 0.8848025192482585
KNN Test Explained Variance for P20_1200ridge : 0.8880122062028821

KNN Test RMSE for Predicted_p20_1200knn : 0.0764088035207859
KNN Test MAE for Predicted_p20_1200knn : 0.0

<ipython-input-98-b5dde1650d2a>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
<ipython-input-98-b5dde1650d2a>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
<ipython-input-98-b5dde1650d2a>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
<ipython-input-98-b5dde1650d2a>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
<ipython-input-98-b5dde1650d2a>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat 

In [99]:
# Calculate the average of all errors
mean_rmse = np.mean(knn_rmse_test)
mean_mae = np.mean(knn_mae_test)
mean_r2 = np.mean(knn_r2_test)
mean_explained_variance = np.mean(knn_explained_variance_test)

# Print the average errors
print("Mean RMSE:", mean_rmse)
print("Mean MAE:", mean_mae)
print("Mean R2 Score:", mean_r2)
print("Mean Explained Variance:", mean_explained_variance)

Mean RMSE: 0.06601863581235817
Mean MAE: 0.0344817789548035
Mean R2 Score: 0.8722347298293763
Mean Explained Variance: 0.8762689091449235


In [100]:
# Test the ANN model
y_pred_ann = best_ann_model.predict(X_test)

# Create a DataFrame to store the test errors
test_error_df = pd.DataFrame(columns=['Target Variable', 'RMSE', 'MAE', 'R2 Score', 'Explained Variance'])

# Calculate the test errors for each target variable
ann_rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_ann, multioutput='raw_values'))
ann_mae_test = mean_absolute_error(y_test, y_pred_ann, multioutput='raw_values')
ann_r2_test = r2_score(y_test, y_pred_ann, multioutput='raw_values')
ann_explained_variance_test = explained_variance_score(y_test, y_pred_ann, multioutput='raw_values')

# Print the test errors for each target variable
for i, target_variable in enumerate(target_variables):
    print("ANN Test RMSE for", target_variable, ":", ann_rmse_test[i])
    print("ANN Test MAE for", target_variable, ":", ann_mae_test[i])
    print("ANN Test R2 Score for", target_variable, ":", ann_r2_test[i])
    print("ANN Test Explained Variance for", target_variable, ":", ann_explained_variance_test[i])
    print()

    # Append the test errors to the DataFrame
    test_error_df = test_error_df.append({
        'Target Variable': target_variable,
        'RMSE': ann_rmse_test[i],
        'MAE': ann_mae_test[i],
        'R2 Score': ann_r2_test[i],
        'Explained Variance': ann_explained_variance_test[i]
    }, ignore_index=True)


# Print the test error DataFrame in a tabular format
print(test_error_df)


ANN Test RMSE for p20_1200LR : 0.04452322222183551
ANN Test MAE for p20_1200LR : 0.03131769756748818
ANN Test R2 Score for p20_1200LR : 0.9327506859225378
ANN Test Explained Variance for p20_1200LR : 0.9327591557815

ANN Test RMSE for p20_1200LIB : 0.050146004002674674
ANN Test MAE for p20_1200LIB : 0.035364475654928576
ANN Test R2 Score for p20_1200LIB : 0.9146924740403
ANN Test Explained Variance for p20_1200LIB : 0.9153384312462497

ANN Test RMSE for P20_1200DT : 0.0480673125820138
ANN Test MAE for P20_1200DT : 0.03620553659225327
ANN Test R2 Score for P20_1200DT : 0.9277637012299417
ANN Test Explained Variance for P20_1200DT : 0.9298757644761207

ANN Test RMSE for P20_1200ridge : 0.07560722281155109
ANN Test MAE for P20_1200ridge : 0.0650751036721339
ANN Test R2 Score for P20_1200ridge : 0.8437429285314929
ANN Test Explained Variance for P20_1200ridge : 0.8879299389257622

ANN Test RMSE for Predicted_p20_1200knn : 0.05762897227759242
ANN Test MAE for Predicted_p20_1200knn : 0.03841

<ipython-input-100-3fbb90b04f28>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
<ipython-input-100-3fbb90b04f28>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
<ipython-input-100-3fbb90b04f28>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
<ipython-input-100-3fbb90b04f28>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
<ipython-input-100-3fbb90b04f28>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.co

In [102]:
# Calculate the average of all errors
mean_rmse = np.mean(ann_rmse_test)
mean_mae = np.mean(ann_mae_test)
mean_r2 = np.mean(ann_r2_test)
mean_explained_variance = np.mean(ann_explained_variance_test)

# Print the average errors
print("Mean RMSE:", mean_rmse)
print("Mean MAE:", mean_mae)
print("Mean R2 Score:", mean_r2)
print("Mean Explained Variance:", mean_explained_variance)


Mean RMSE: 0.05456447370776986
Mean MAE: 0.04075980639317094
Mean R2 Score: 0.9109049921888885
Mean Explained Variance: 0.9233322922900377


In [103]:
# Test the Elastic Net model
y_pred_en = best_en_model.predict(X_test)

# Create a DataFrame to store the test errors
test_error_df = pd.DataFrame(columns=['Target Variable', 'RMSE', 'MAE', 'R2 Score', 'Explained Variance'])

# Calculate the test errors for each target variable
en_rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_en, multioutput='raw_values'))
en_mae_test = mean_absolute_error(y_test, y_pred_en, multioutput='raw_values')
en_r2_test = r2_score(y_test, y_pred_en, multioutput='raw_values')
en_explained_variance_test = explained_variance_score(y_test, y_pred_en, multioutput='raw_values')

# Print the test errors for each target variable
for i, target_variable in enumerate(target_variables):
    print("Elastic Net Test RMSE for", target_variable, ":", en_rmse_test[i])
    print("Elastic Net Test MAE for", target_variable, ":", en_mae_test[i])
    print("Elastic Net Test R2 Score for", target_variable, ":", en_r2_test[i])
    print("Elastic Net Test Explained Variance for", target_variable, ":", en_explained_variance_test[i])
    print()

    # Append the test errors to the DataFrame
    test_error_df = test_error_df.append({
        'Target Variable': target_variable,
        'RMSE': en_rmse_test[i],
        'MAE': en_mae_test[i],
        'R2 Score': en_r2_test[i],
        'Explained Variance': en_explained_variance_test[i]
    }, ignore_index=True)



# Print the test error DataFrame in a tabular format
print(test_error_df)


Elastic Net Test RMSE for p20_1200LR : 0.09966643768559182
Elastic Net Test MAE for p20_1200LR : 0.05984428937192464
Elastic Net Test R2 Score for p20_1200LR : 0.6630134590022398
Elastic Net Test Explained Variance for p20_1200LR : 0.6801236320222581

Elastic Net Test RMSE for p20_1200LIB : 0.09966643760017549
Elastic Net Test MAE for p20_1200LIB : 0.05984428933871916
Elastic Net Test R2 Score for p20_1200LIB : 0.6630134589426948
Elastic Net Test Explained Variance for p20_1200LIB : 0.6801236319498722

Elastic Net Test RMSE for P20_1200DT : 0.09742169331832459
Elastic Net Test MAE for P20_1200DT : 0.06194208258954672
Elastic Net Test R2 Score for P20_1200DT : 0.7032661098723189
Elastic Net Test Explained Variance for P20_1200DT : 0.7200990417750746

Elastic Net Test RMSE for P20_1200ridge : 0.0970886759652253
Elastic Net Test MAE for P20_1200ridge : 0.0659673130491312
Elastic Net Test R2 Score for P20_1200ridge : 0.742338063958337
Elastic Net Test Explained Variance for P20_1200ridge :

<ipython-input-103-ca28959c913e>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
<ipython-input-103-ca28959c913e>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
<ipython-input-103-ca28959c913e>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
<ipython-input-103-ca28959c913e>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
<ipython-input-103-ca28959c913e>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.co

In [104]:
# Calculate the average of all errors
mean_rmse = np.mean(en_rmse_test)
mean_mae = np.mean(en_mae_test)
mean_r2 = np.mean(en_r2_test)
mean_explained_variance = np.mean(en_explained_variance_test)

# Print the average errors
print("Mean RMSE:", mean_rmse)
print("Mean MAE:", mean_mae)
print("Mean R2 Score:", mean_r2)
print("Mean Explained Variance:", mean_explained_variance)

Mean RMSE: 0.10001795267371649
Mean MAE: 0.06411103733795431
Mean R2 Score: 0.7056491893648056
Mean Explained Variance: 0.7227296222984426


In [105]:
# Test the Decision Tree model
y_pred_dt = best_dt_model.predict(X_test)

# Create a DataFrame to store the test errors
test_error_df = pd.DataFrame(columns=['Target Variable', 'RMSE', 'MAE', 'R2 Score', 'Explained Variance'])

# Calculate the test errors for each target variable
dt_rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_dt, multioutput='raw_values'))
dt_mae_test = mean_absolute_error(y_test, y_pred_dt, multioutput='raw_values')
dt_r2_test = r2_score(y_test, y_pred_dt, multioutput='raw_values')
dt_explained_variance_test = explained_variance_score(y_test, y_pred_dt, multioutput='raw_values')

# Print the test errors for each target variable
for i, target_variable in enumerate(target_variables):
    print("Decision Tree Test RMSE for", target_variable, ":", dt_rmse_test[i])
    print("Decision Tree Test MAE for", target_variable, ":", dt_mae_test[i])
    print("Decision Tree Test R2 Score for", target_variable, ":", dt_r2_test[i])
    print("Decision Tree Test Explained Variance for", target_variable, ":", dt_explained_variance_test[i])
    print()

    # Append the test errors to the DataFrame
    test_error_df = test_error_df.append({
        'Target Variable': target_variable,
        'RMSE': dt_rmse_test[i],
        'MAE': dt_mae_test[i],
        'R2 Score': dt_r2_test[i],
        'Explained Variance': dt_explained_variance_test[i]
    }, ignore_index=True)


# Print the test error DataFrame in a tabular format
print(test_error_df)


Decision Tree Test RMSE for p20_1200LR : 0.09093595199038995
Decision Tree Test MAE for p20_1200LR : 0.05165759648256419
Decision Tree Test R2 Score for p20_1200LR : 0.7194657316713158
Decision Tree Test Explained Variance for p20_1200LR : 0.7200649492882851

Decision Tree Test RMSE for p20_1200LIB : 0.09093595194183467
Decision Tree Test MAE for p20_1200LIB : 0.05165759644932879
Decision Tree Test R2 Score for p20_1200LIB : 0.7194657314404805
Decision Tree Test Explained Variance for p20_1200LIB : 0.7200649490621379

Decision Tree Test RMSE for P20_1200DT : 0.09108400505642839
Decision Tree Test MAE for P20_1200DT : 0.05384096238007944
Decision Tree Test R2 Score for P20_1200DT : 0.7406178774679406
Decision Tree Test Explained Variance for P20_1200DT : 0.7413126098316407

Decision Tree Test RMSE for P20_1200ridge : 0.09369705536407544
Decision Tree Test MAE for P20_1200ridge : 0.058388578141806494
Decision Tree Test R2 Score for P20_1200ridge : 0.7600255562837178
Decision Tree Test Ex

<ipython-input-105-e448b12d11a7>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
<ipython-input-105-e448b12d11a7>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
<ipython-input-105-e448b12d11a7>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
<ipython-input-105-e448b12d11a7>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
<ipython-input-105-e448b12d11a7>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.co

In [106]:
# Calculate the average of all errors
mean_rmse = np.mean(dt_rmse_test)
mean_mae = np.mean(dt_mae_test)
mean_r2 = np.mean(dt_r2_test)
mean_explained_variance = np.mean(dt_explained_variance_test)

# Print the average errors
print("Mean RMSE:", mean_rmse)
print("Mean MAE:", mean_mae)
print("Mean R2 Score:", mean_r2)
print("Mean Explained Variance:", mean_explained_variance)


Mean RMSE: 0.09350863885081794
Mean MAE: 0.0558565496744921
Mean R2 Score: 0.7436139589725271
Mean Explained Variance: 0.7442163617443817


In [107]:
# Test the RANSAC model
y_pred_ransac = best_ransac_model.predict(X_test)

# Create a DataFrame to store the test errors
test_error_df = pd.DataFrame(columns=['Target Variable', 'RMSE', 'MAE', 'R2 Score', 'Explained Variance'])

# Calculate the test errors for each target variable
ransac_rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_ransac, multioutput='raw_values'))
ransac_mae_test = mean_absolute_error(y_test, y_pred_ransac, multioutput='raw_values')
ransac_r2_test = r2_score(y_test, y_pred_ransac, multioutput='raw_values')
ransac_explained_variance_test = explained_variance_score(y_test, y_pred_ransac, multioutput='raw_values')

# Print the test errors for each target variable
for i, target_variable in enumerate(target_variables):
    print("RANSAC Test RMSE for", target_variable, ":", ransac_rmse_test[i])
    print("RANSAC Test MAE for", target_variable, ":", ransac_mae_test[i])
    print("RANSAC Test R2 Score for", target_variable, ":", ransac_r2_test[i])
    print("RANSAC Test Explained Variance for", target_variable, ":", ransac_explained_variance_test[i])
    print()

    # Append the test errors to the DataFrame
    test_error_df = test_error_df.append({
        'Target Variable': target_variable,
        'RMSE': ransac_rmse_test[i],
        'MAE': ransac_mae_test[i],
        'R2 Score': ransac_r2_test[i],
        'Explained Variance': ransac_explained_variance_test[i]
    }, ignore_index=True)


# Print the test error DataFrame in a tabular format
print(test_error_df)


RANSAC Test RMSE for p20_1200LR : 0.04635484227977966
RANSAC Test MAE for p20_1200LR : 0.0319860731697534
RANSAC Test R2 Score for p20_1200LR : 0.9271037983141678
RANSAC Test Explained Variance for p20_1200LR : 0.9280422667796939

RANSAC Test RMSE for p20_1200LIB : 0.046354842196923075
RANSAC Test MAE for p20_1200LIB : 0.03198607311435073
RANSAC Test R2 Score for p20_1200LIB : 0.9271037984369352
RANSAC Test Explained Variance for p20_1200LIB : 0.9280422669030661

RANSAC Test RMSE for P20_1200DT : 0.04448249810674156
RANSAC Test MAE for P20_1200DT : 0.030975811137911136
RANSAC Test R2 Score for P20_1200DT : 0.9381365494130379
RANSAC Test Explained Variance for P20_1200DT : 0.9388661635671339

RANSAC Test RMSE for P20_1200ridge : 0.044555742187805836
RANSAC Test MAE for P20_1200ridge : 0.03121484061243194
RANSAC Test R2 Score for P20_1200ridge : 0.945734916778634
RANSAC Test Explained Variance for P20_1200ridge : 0.9462568495026018

RANSAC Test RMSE for Predicted_p20_1200knn : 0.05564470

<ipython-input-107-4e72c5fee130>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
<ipython-input-107-4e72c5fee130>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
<ipython-input-107-4e72c5fee130>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
<ipython-input-107-4e72c5fee130>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
<ipython-input-107-4e72c5fee130>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.co

In [108]:

# Calculate the average of all errors
mean_rmse = np.mean(ransac_rmse_test)
mean_mae = np.mean(ransac_mae_test)
mean_r2 = np.mean(ransac_r2_test)
mean_explained_variance = np.mean(ransac_explained_variance_test)

# Print the average errors
print("Mean RMSE:", mean_rmse)
print("Mean MAE:", mean_mae)
print("Mean R2 Score:", mean_r2)
print("Mean Explained Variance:", mean_explained_variance)

Mean RMSE: 0.04665669706928063
Mean MAE: 0.032344157627124566
Mean R2 Score: 0.9358862562934869
Mean Explained Variance: 0.9366854945355304


In [109]:
# Test the GPR model
y_pred_gpr = best_gpr_model.predict(X_test)

# Create a DataFrame to store the test errors
test_error_df = pd.DataFrame(columns=['Target Variable', 'RMSE', 'MAE', 'R2 Score', 'Explained Variance'])

# Calculate the test errors for each target variable
gpr_rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_gpr, multioutput='raw_values'))
gpr_mae_test = mean_absolute_error(y_test, y_pred_gpr, multioutput='raw_values')
gpr_r2_test = r2_score(y_test, y_pred_gpr, multioutput='raw_values')
gpr_explained_variance_test = explained_variance_score(y_test, y_pred_gpr, multioutput='raw_values')

# Print the test errors for each target variable
for i, target_variable in enumerate(target_variables):
    print("GPR Test RMSE for", target_variable, ":", gpr_rmse_test[i])
    print("GPR Test MAE for", target_variable, ":", gpr_mae_test[i])
    print("GPR Test R2 Score for", target_variable, ":", gpr_r2_test[i])
    print("GPR Test Explained Variance for", target_variable, ":", gpr_explained_variance_test[i])
    print()

    # Append the test errors to the DataFrame
    test_error_df = test_error_df.append({
        'Target Variable': target_variable,
        'RMSE': gpr_rmse_test[i],
        'MAE': gpr_mae_test[i],
        'R2 Score': gpr_r2_test[i],
        'Explained Variance': gpr_explained_variance_test[i]
    }, ignore_index=True)


# Print the test error DataFrame in a tabular format
print(test_error_df)


GPR Test RMSE for p20_1200LR : 0.043291373684835883
GPR Test MAE for p20_1200LR : 0.028886752742653187
GPR Test R2 Score for p20_1200LR : 0.9364204547173266
GPR Test Explained Variance for p20_1200LR : 0.9366366159825118

GPR Test RMSE for p20_1200LIB : 0.043291373579166646
GPR Test MAE for p20_1200LIB : 0.028886752667991206
GPR Test R2 Score for p20_1200LIB : 0.9364204549074947
GPR Test Explained Variance for p20_1200LIB : 0.9366366161736864

GPR Test RMSE for P20_1200DT : 0.04083056118016468
GPR Test MAE for P20_1200DT : 0.027696480390037664
GPR Test R2 Score for P20_1200DT : 0.9478773485126538
GPR Test Explained Variance for P20_1200DT : 0.9480321754089279

GPR Test RMSE for P20_1200ridge : 0.039917250960175094
GPR Test MAE for P20_1200ridge : 0.027572295343596644
GPR Test R2 Score for P20_1200ridge : 0.956445367101613
GPR Test Explained Variance for P20_1200ridge : 0.9565489450744278

GPR Test RMSE for Predicted_p20_1200knn : 0.05241816361178905
GPR Test MAE for Predicted_p20_1200k

<ipython-input-109-8b38e9d9ef4c>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
<ipython-input-109-8b38e9d9ef4c>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
<ipython-input-109-8b38e9d9ef4c>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
<ipython-input-109-8b38e9d9ef4c>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_error_df = test_error_df.append({
<ipython-input-109-8b38e9d9ef4c>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.co

In [111]:
# Calculate the average of all errors
mean_rmse = np.mean(gpr_rmse_test)
mean_mae = np.mean(gpr_mae_test)
mean_r2 = np.mean(gpr_r2_test)
mean_explained_variance = np.mean(gpr_explained_variance_test)

# Print the average errors
print("Mean RMSE:", mean_rmse)
print("Mean MAE:", mean_mae)
print("Mean R2 Score:", mean_r2)
print("Mean Explained Variance:", mean_explained_variance)


Mean RMSE: 0.042782991851951224
Mean MAE: 0.02887437273252232
Mean R2 Score: 0.9458431953333637
Mean Explained Variance: 0.9460963030172926
